In [3]:
# __future__ import should always be first
from __future__ import annotations

# Standard library imports
from collections import defaultdict

# Third-party imports
import torch
import numpy as np
import torch.nn as nn
from torch.distributions.categorical import Categorical

# Gymnasium & Minigrid imports
import gymnasium as gym  # Correct way to import Gymnasium
from minigrid.core.constants import COLOR_NAMES
from minigrid.core.constants import DIR_TO_VEC
from minigrid.core.grid import Grid
from minigrid.core.actions import Actions
from minigrid.core.mission import MissionSpace
from minigrid.core.world_object import Door, Goal, Key, Wall
from minigrid.manual_control import ManualControl
from minigrid.minigrid_env import MiniGridEnv
from gymnasium.utils.play import play
import pandas as pd
# Visualization imports
import matplotlib.pyplot as plt


# Environment Set Up

# 3 Action with Directions

In [4]:

class SimpleEnv(MiniGridEnv):
    def __init__(
            self, 
            size=10, 
            agent_start_pos=(1, 8), 
            agent_start_dir=0, 
            max_steps=256, 
            **kwargs,
    ):
        self.agent_start_pos = agent_start_pos
        self.agent_start_dir = agent_start_dir
        self.goal_pos = (8, 1)
        
        
        
        mission_space = MissionSpace(mission_func=self._gen_mission)

        super().__init__(
            mission_space=mission_space,
            grid_size=size,
            max_steps=max_steps,
            **kwargs,
        )

        self.action_space = gym.spaces.Discrete(3)
    @staticmethod
    def _gen_mission():
        return "Find the shortest path"

    def _gen_grid(self, width, height):
        #create gird
        self.grid = Grid(width, height)
        #place barrier
        self.grid.wall_rect(0, 0, width, height)
        #place goal
        self.put_obj(Goal(), 8, 1)
        #place walls
        for i in range(1, width // 2):
            self.grid.set(i, width - 4, Wall())
            self.grid.set(i + width // 2 - 1, width - 7, Wall())
        #place agent
        if self.agent_start_pos is not None:
            self.agent_pos = self.agent_start_pos #check this
            self.agent_dir = self.agent_start_dir
        else:
            self.place_agent()

        self.mission = "find the shortest path"
    
    def count_states(self):
        free_cells = sum(1 for x in range(self.grid.width)
                      for y in range(self.grid.height)
                      if not self.grid.get(x, y)) * 4
        return free_cells 


# Manual Environment Testing

In [3]:
env = SimpleEnv(render_mode="human") 


In [7]:
env = SimpleEnv(render_mode="human")  # Create environment
# free_energy_solver = FreeEnergyMin(env, beta=0.5)
obs = env.reset()[0]  # Reset environment and get initial state
env.render()  # Display initial state


In [ ]:
env.count_states()

In [11]:
obs, reward, done, truncated, info = env.step(0)  #turn left
env.render()

In [111]:
obs, reward, done, truncated, info = env.step(1) #turn right
env.render()

In [123]:
obs, reward, done, truncated, info = env.step(2) #move forward
env.render()

In [ ]:
free_energy_solver.position_to_state_index(obs)

In [ ]:
free_energy_solver.state_index_to_position(328)

In [24]:
#test env
env = SimpleEnv(render_mode="human") 
# env.reset()
# free_energy_solver = FreeEnergyMin(env, beta=0.5)
# free_energy_solver.estimate_transitions() #for when you want to do a random walk, no learning


# Define a Policy Based On Information-to-Go Method

In [ ]:
env.action_space.n * env.count_states() #number of actions * number of states

In [ ]:
env.count_states()

In [ ]:
free_energy_solver.num_states

In [ ]:
free_energy_solver.num_actions

In [ ]:
env.agent_pos

In [ ]:
free_energy_solver.num_states

In [88]:
class FreeEnergyMin:
    """Free Energy Minimization to find optimal policy in MiniGrid Environment."""

    def __init__(
            self, 
            env, 
            beta=0.5
    ):
        
        self.env = env  # MiniGrid environment
        self.num_states = np.int64(env.count_states())  # Number of states
        print(f"Number of states: {self.num_states}")
        self.num_actions = env.action_space.n  # 3 actions (Forward, Left+Forward, Right+Forward)
        print(f"Number of actions: {self.num_actions}")
        self.beta = beta  # Temperature parameter

        # Initialize policy (uniform distribution)
        self.Pi_a_s = np.full((self.num_states, self.num_actions), 1 / self.num_actions) 
        print(f"policy shape: {self.Pi_a_s.shape}")
        self.Pi_a = np.full(self.num_actions, 1 / self.num_actions)
        print(f"marginal action distribution shape: {self.Pi_a.shape}")
        

        
    
    def position_to_state_index(self, state):
        """Converts (x, y, direction) into a unique state index."""
        grid_width = self.env.grid.width
        grid_height = self.env.grid.height
        x, y = self.env.agent_pos  
        direction = self.env.agent_dir  

        return np.int64((y * grid_width + x) * 4 + direction)

    def state_index_to_position(self, state_idx):
        """Converts a 1D state index back into (x, y, direction)."""
        grid_width = self.env.grid.width
        
        direction = state_idx % 4
        linear_idx = state_idx // 4

        y, x = divmod(linear_idx, grid_width)  # Convert to (x, y)
        
        return x, y, direction
    
    def estimate_transitions(self, episodes=10):
        """Simulate environment for debugging with random agent movements."""
        for _ in range(1, episodes + 1):
            state, _ = self.env.reset()  
            

            for step in range(self.env.max_steps):
                action = self.env.action_space.sample() 
                print(f"Step {step}: Action {action} taken.")
                next_state, _, done, _, _ = self.env.step(action)  
                
                #Convert states to indices for debugging
                s_idx = self.position_to_state_index(state)
                s_next_idx = self.position_to_state_index(next_state)
                
                state = next_state  #Update current state
                
                if done:
                    
                    break  # Stop if episode ends

        print("Finished testing environment.")

    def compute_free_energy(self, num_iterations=500):
        """Iteratively update Free Energy and optimize the policy."""
        states = self.num_states
        actions = self.num_actions

        print(f'Number of states: {states}')
        print(f'Number of actions: {actions}')

        # Initialize transition probabilities (deterministic)
        P_s_given_sa = np.zeros((self.num_states, self.num_actions, self.num_states))

        # Assume uniform state distribution initially
        P_s = np.full(self.num_states, 1 / self.num_states)

        # Initialize Free Energy
        self.F = np.zeros((self.num_states, self.num_actions))

        free_energy = []

        for iteration in range(num_iterations):
            prev_F = np.sum(self.F)

            for state_idx in range(states):  # Loop over all states
                x, y, direction = self.state_index_to_position(state_idx)

                # Ensure the agent is placed in a valid position
                if self.env.grid.get(x, y) is not None and not isinstance(self.env.grid.get(x, y), Goal):
                    print(f"Invalid state {x}, {y} (occupied). Trying another.")
                    continue
                    

                self.env.place_agent((x, y))
                self.env.agent_dir = direction
                

                for a in range(actions):  # Loop over actions
                    print(f"Processing state {state_idx}, action {a}")

                    # Execute action
                    action = np.argmax(self.Pi_a_s[state_idx])  # Select action based on policy
                    next_state, _, done, _, _ = self.env.step(action)
                    s_next = self.position_to_state_index(next_state) #convert position to next state

                    # If terminal, break
                    if done:
                        break

                    # Sample next action from policy
                    next_action = np.argmax(self.Pi_a_s[s_next])
                    next_state_1, _, _, _, _ = self.env.step(next_action)
                    s_next_next = self.position_to_state_index(next_state_1)

                    # **Deterministic Transition Probability**
                    P_s_given_sa[state_idx, action, s_next] = 1

                    # Compute expectation term (log ratio of policy)
                    J = np.sum(np.log(np.maximum(self.Pi_a_s[s_next, :] / np.maximum(self.Pi_a, 1e-10), 1e-10)))

                    # Define reward function
                    reward = 0 if self.env.reached_goal else -1

                    # Update Free Energy functional
                    self.F[state_idx, a] = (
                        np.sum(P_s_given_sa[state_idx, a, :] * np.log(np.maximum(P_s_given_sa[state_idx, a, :] / P_s, 1e-10)))
                        - self.beta * reward + J
                    )

            # Compute partition function Zπ(s, β)
            Z = np.sum(self.Pi_a[None, :] * np.exp(-self.F / (self.beta + 1e-10)), axis=1, keepdims=True) + 1e-5

            # Update policy π(a|s)
            self.Pi_a_s = (self.Pi_a[None, :] / Z) * np.exp(-self.F)

            # Update marginal π(a)
            self.Pi_a = np.sum(self.Pi_a_s * P_s[:, None], axis=0) + 1e-10
            self.Pi_a /= np.sum(self.Pi_a)

            # Compute max policy change
            max_policy_change = np.max(np.abs(self.Pi_a_s - self.Pi_a[None, :]))

            # Print status
            print(f"Iteration {iteration}: Free Energy Sum: {np.sum(self.F)}, Change: {prev_F - np.sum(self.F)}")
            print(f"Iteration {iteration}: Max Policy Change = {max_policy_change}")

            free_energy.append(np.sum(self.F))
            
            # Convergence check
            if max_policy_change < 1e-5:
                print(f"Converged at iteration {iteration}.")
                break
            
            
        
        policy = self.Pi_a_s

        return policy, free_energy 


    def run_policy(self, policy):
        """Run the environment using the learned policy. Convert state in policy to its position using """
        
        start_pos = self.env.reset()[0] #reset the environment and get initial state
        state_idx = self.position_to_state_index(start_pos) #convert starting position to index
        done = False
        while not done:
            x, y, direction = self.state_index_to_position(state_idx) 
            self.env.place_agent((x, y))
            self.env.agent_dir = direction
            action = np.argmax(policy[state_idx])  # Choose best action from policy
            position, _, done, _, _ = self.env.step(action) #take the action
            state_idx = self.position_to_state_index(position) #use this next position , decode the state
            self.env.render()
            


    def plot_free_energy(self, free_energy):
        """Plot the free energy over iterations."""
        

        plt.figure(figsize=(8, 5))
        plt.plot(free_energy, marker='o', linestyle='-')
        plt.xlabel("Iteration")
        plt.ylabel("Sum of Free Energy")
        plt.title("Free Energy Minimization Over Iterations")
        plt.grid()
        plt.show()

# Q learning

assume that -bR + F(a,s,b) is basically like Q learning

In [ ]:
env.count_states()

In [ ]:
(env.width-2) * (env.height-2) * 4 - (8 *4) 

In [15]:
class Qlearning:
    def __init__(
        self,
        env,
        learning_rate = 0.9,
        discount_factor = 0.9,
        epsilon = 0.5,
        epochs = 1000
         
    ):
        self.env = env # MiniGrid environment called from the class Minigrid
        self.num_states = np.int64(env.count_states())
        self.num_actions = env.action_space.n
        self.Q_table = np.zeros((((env.width -2) * (env.height -2) *4), self.num_actions)) # Q table has goal states in it as well
        # print(f"Q table shape: {self.Q_table.shape}")
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor # discount factor
        self.epsilon =  epsilon # exploration probability
        self.epochs = epochs


        self.allowed_state_idx = self.find_state_indexes(env)
        # print(f"allowed state indexes: {self.allowed_state_idx}")


    
    def position_to_state_index(self, tuple_position = None): 
        grid_width = self.env.grid.width -2
        if tuple_position is None:
            direction = self.env.agent_dir
            x, y = self.env.agent_pos

        else:
            if not isinstance(tuple_position, tuple) or len(tuple_position) != 3:
                raise ValueError(f"Invalid position format: {tuple_position}")
            x, y, direction = tuple_position
        return np.int64(((y-1) * grid_width + (x-1)) * 4 + direction) 
    
    def state_index_to_position(self, state_idx):
        """Converts a scalar state index back into (x, y, direction)."""
        grid_width = self.env.grid.width-2
        
        direction = state_idx % 4
        linear_idx = state_idx // 4

        y, x = divmod(linear_idx, grid_width)  # Convert to (x, y)
        
        return x+1, y+1, direction
    
    def find_state_indexes(self, env):
        """Counts all states except walls and barriers"""
        state_indexes_list = []
        for x in range(1, env.grid.width-1):
            for y in range(1, env.grid.height-1):
                if env.grid.get(x, y) is None: #grabs all empty spaces
                    for direction in range(4):
                        state_index = self.position_to_state_index((x, y, direction))
                        state_indexes_list.append(state_index)
        return state_indexes_list        
    
    
    def train(self, epochs):
        goal_states = [self.position_to_state_index((8, 1, d)) for d in range(4)] #goal state index
        
        for epoch in range(epochs):
            current_state = np.random.choice(self.allowed_state_idx)
            x,y,dir = self.state_index_to_position(current_state)
            self.env.agent_pos = (x,y)
            self.env.agent_dir = dir
            print(f"Epoch {epoch}: Starting state: {current_state}")
            print(f"Epoch {epoch}: Starting position: {self.state_index_to_position(current_state)}")

            while current_state not in goal_states:
                # #replace current index with its table index
                # table_index = self.state_to_index[current_state]
                # Epsilon-greedy action selection
                if np.random.rand() < self.epsilon:
                    action = self.env.action_space.sample()
                    print(f"random action: {action}, exploring")
                else:
                    action = np.argmax(self.Q_table[current_state])
                    print(f"greedy action: {action}, exploiting")


                #transition to the next state
                next_obs, _, done, _, _ = self.env.step(action)

                # print(f"next_observation: {next_obs}")
                next_state = self.position_to_state_index()
                print(f"next_state: {next_state}")
                print(f"next  step: Agent Pos: {self.env.agent_pos}, Dir: {self.env.agent_dir}")
                #reward (-1 for each step thats not the gaol)
                reward = 0 if next_state in goal_states else -1

                self.Q_table[current_state, action] += self.learning_rate * (reward + self.discount_factor * np.max(self.Q_table[next_state]) - self.Q_table[current_state, action])
                #update state
                if next_state in goal_states:
                    break
                assert(False)
                current_state = next_state
        
        
    

    def run_policy_2(self):
        """Run the environment using the learned policy from a Q-table."""

        self.env.reset()[0]  # Reset environment
        current_state = self.position_to_state_index()  # Convert starting position to index
        done = False
        step_count = 0  # Track steps to prevent infinite loops

        while not done and step_count < 100:  # Prevent infinite loops
            # Debugging: Print current state and Q-values
            print(f"Step {step_count}: State {current_state}")
            print(f"Q-values: {self.Q_table[current_state]}")
            
            action = np.argmax(self.Q_table[current_state])  # Choose best action
            print(f"Chosen action: {action}")

            # Before taking the step, print agent's current position
            print(f"Before step: Agent Pos: {self.env.agent_pos}, Dir: {self.env.agent_dir}")

            next_obs, _, done, _, _ = self.env.step(action)  # Take action

            # After step, print agent's new position
            print(f"After step: Agent Pos: {self.env.agent_pos}, Dir: {self.env.agent_dir}")

            next_state = self.position_to_state_index()  # Convert new state

            # Detect if the agent is looping in the same state
            if next_state == current_state:
                print(f"⚠️ Warning: Agent is stuck! Current state {current_state} is the same as next state {next_state}.")
                break  # Prevent infinite loop

            self.env.render()  # Visualize movement
            current_state = next_state  # Update current state
            step_count += 1





In [16]:
# Convert Q-table into a DataFrame for better readability
q_table_df = pd.DataFrame(q_training.Q_table, columns=[f"Action {i}" for i in range(3)])

# Add a column for the corresponding grid position
positions = [q_training.state_index_to_position(state_idx) for state_idx in range(q_training.Q_table.shape[0])]
q_table_df["Grid Position"] = positions  # Append positions to Q-table


In [ ]:
q_table_df

In [ ]:
Q_table_correct_800 = q_training.Q_table
%store Q_table_correct_800

In [ ]:
q_training.state_index_to_position()

In [17]:
env = SimpleEnv(render_mode=None) 
env.reset()
q_training = Qlearning(env)
q_training.train(50)


Epoch 0: Starting state: 110
Epoch 0: Starting position: (np.int64(4), np.int64(4), np.int64(2))
greedy action: 0, exploiting
next_state: 109
next  step: Agent Pos: (np.int64(4), np.int64(4)), Dir: 1


AssertionError: 

In [ ]:
Q_table_epochs_600000_e_95_discount_9_learnrate_9= q_training.Q_table
%store Q_table_epochs_600000_e_95_discount_9_learnrate_9

In [14]:
obs, reward, done, truncated, info = env.step(0)

In [ ]:
q_training.position_to_state_index()

In [ ]:
env.grid.get(1,6)

In [ ]:
env.agent_pos

In [ ]:
env.agent_dir

In [ ]:
env_human = SimpleEnv(render_mode="human") #make same env but in human mode 

q_training.env= env_human #switch out the env in q training with human env
q_training.run_policy_2() #run the policy in the human env

In [43]:
env_human.close()

In [ ]:
q_training.state_indexes_list

In [ ]:
for i, state in enumerate(q_training.state_indexes_list):
    x, y, _ = q_training.state_index_to_position(state) 
    print(f"State index {i}: Position ({x}, {y})")

In [ ]:
Q_table = q_training.Q_table
%store Q_table

In [10]:
Q_table = q_training.Q_table
def state_index_to_position(state_idx):
    """Converts a scalar state index back into (x, y, direction)."""
    grid_width = 8
        
    direction = state_idx % 4
    linear_idx = state_idx // 4

    y, x = divmod(linear_idx, grid_width)  # Convert to (x, y)
        
    return x+1, y+1, direction

# Q Learning with Policy Updates each Epoch

In [444]:
class Qlearning_frompolicy:
    def __init__(
        self,
        env,
        epochs = 1000,
        beta = 5,
        deterministic = True
         
    ):
        #eventually need to replace num_states with the number of states in the environment formally
        self.env = env # MiniGrid environment called from the class Minigrid
        self.num_states = ((env.width -2) * (env.height -2) *4)
        self.num_actions = env.action_space.n
        self.deterministic = deterministic
        self.Free_energy_table = np.zeros((self.num_states, self.num_actions)) # Q table has goal states in it as well
        print(f"Free energy table shape: {self.Free_energy_table.shape}")
        self.epochs = epochs
        self.Pi_a = np.full(self.num_actions, 1/self.num_actions)
        self.Pi_a_s = np.full((self.num_states, self.num_actions), 1 / self.num_actions)
        print(f"policy shape: {self.Pi_a_s.shape}")
        self.P_s_by_s = np.zeros((self.num_states, self.num_states)) # helps caluclate P(s'|s), to then calculate P(s)
        print(f"s by s shape: {self.P_s_by_s.shape}")
        self.allowed_state_idx = self.find_state_indexes(env)
        self.steps = 0 #within each epoch, this counts the number of state transitions
        self.Zeta = np.zeros(self.num_states)
        self.P_s = np.zeros(self.num_states)
        self.P_s_tplus1 = np.zeros(self.num_states)
        self.P_s_given_s_a = np.zeros((self.num_states, self.num_actions, self.num_states))
        self.beta = beta
        self.Pi_a_tplus1 = np.zeros(self.num_actions)

        # print(f"allowed state indexes: {self.allowed_state_idx}")


    
    def position_to_state_index(self, tuple_position = None): 
        grid_width = self.env.grid.width -2
        if tuple_position is None:
            direction = self.env.agent_dir
            x, y = self.env.agent_pos

        else:
            if not isinstance(tuple_position, tuple) or len(tuple_position) != 3:
                raise ValueError(f"Invalid position format: {tuple_position}")
            x, y, direction = tuple_position
        return np.int64(((y-1) * grid_width + (x-1)) * 4 + direction) 
    
    def state_index_to_position(self, state_idx):
        """Converts a scalar state index back into (x, y, direction)."""
        grid_width = self.env.grid.width-2
        
        direction = state_idx % 4
        linear_idx = state_idx // 4

        y, x = divmod(linear_idx, grid_width)  # Convert to (x, y)
        
        return x+1, y+1, direction
    
    
    def find_state_indexes(self, env):
        """Counts all states except walls and barriers"""
        state_indexes_list = []
        for x in range(1, env.grid.width-1):
            for y in range(1, env.grid.height-1):
                if env.grid.get(x, y) is None: #grabs all empty spaces
                    for direction in range(4):
                        state_index = self.position_to_state_index((x, y, direction))
                        state_indexes_list.append(state_index)
        return state_indexes_list        
    
    def next_state_index(self, current_state_idx, action):
        # Convert current state index into (x, y, direction)
        x, y, direction = self.state_index_to_position(current_state_idx )
        
        if action == 0:  # Turn left
            direction = (direction - 1) % 4
        elif action == 1:  # Turn right
            direction = (direction + 1) % 4
        elif action == 2:  # Move forward
            if direction == 0:   # looking right increase x
                x += 1
            elif direction == 1: # looking down increase y
                y += 1
            elif direction == 2: # facing left: decrease x
                x -= 1
            elif direction == 3: # looking up: decrease y
                y -= 1
        else:
            raise ValueError(f"Invalid action: {action}")
        
        # Convert the new (x, y, direction) back to a state index.
        next_state = self.position_to_state_index((x, y, direction))
        if next_state not in self.allowed_state_idx:
            # If the next state is not allowed, return the current state
            return current_state_idx
        return next_state
    
    def find_all_next_states(self):
        """Find all possible next states for each state and action."""
        for state in self.allowed_state_idx:
            for action in range(self.num_actions):
                next_state = self.next_state_index(state, action)
                self.P_s_given_s_a[state, action, next_state] = 1
                # print(f"State {state}, Action {action} leads to State {next_state}")

    def find_connected_states(self):
        """
        Build and return a connectivity matrix P_s_by_s where the element at [state, next_state]
        is given by the probability from self.Pi_a_s for the action that leads from state to next_state.
        """
        # Loop over all states.
        for state in self.allowed_state_idx:
            # Loop over all actions for the state.
            for action in range(self.num_actions):
                # Find the next state: assume a deterministic transition where exactly one entry is 1.
                next_state = np.argmax(self.P_s_given_s_a[state, action, :])
                # Set the connectivity matrix: you might choose to sum if multiple actions lead to the same state.
                self.P_s_by_s[state, next_state] = self.Pi_a_s[state, action]
        return self.P_s_by_s
    
    
    
    
    # def find_connected_states(self):
    #     for state in range(self.P_s_given_s_a.shape[0]):
    #         next_state, action = self.P_s_given_s_a[state]
    #         self.P_s_by_s[state, next_state] = self.Pi_a_s[state, action]  # Store the action and its probability
    #     return self.P_s_by_s
    
    # def connected_states(self):  #action that takes you there and its probability 
    #     """Check if two states are connected and return the action and probability."""
        
    #     for state1 in self.allowed_state_idx:
            
    #         for state2 in self.allowed_state_idx:
                
    #             if state1 != state2:
    #                 # Check if the two states are connected
    #                 for action in range(self.num_actions):
    #                     # Check if taking action i from state1 leads to state2
                        
                        
    #                     if next_state == state2:
    #                         self.P_s_by_s[state1, state2] = self.Pi_a_s[state1, action]  # Store the action and its probability
    #     return self.P_s_by_s            
            
    # def populate_P_s_given_s_a(self):
    #     for state in self.allowed_state_idx:
    #         for action in range(self.num_actions):
    #             env.reset()
    #             x, y, direction = self.state_index_to_position(state)
    #             self.env.place_agent((x, y))
    #             self.env.agent_dir = direction
    #             next_obs, _, done, _, _ = self.env.step(action)
    #             next_state = self.position_to_state_index()
    #             self.P_s_given_s_a[state, action, next_state] = 1




    def train(self, epochs):
        tol = 1e-5
        self.find_all_next_states() #generates P(s'|s,a) matrix
        self.find_connected_states() #call this to initialize the P(s'|s) matrix
        goal_states = [self.position_to_state_index((8, 1, d)) for d in range(4)] #goal state index
         
        

        # print(f'P_s_by_s matrix: {self.P_s_by_s}')


        for epoch in range(epochs):
            current_state = np.random.choice(self.allowed_state_idx)
            x, y,dir = self.state_index_to_position(current_state)
        
            self.env.agent_pos = (x,y)
            self.env.agent_dir = dir
            print(f'current state position: {self.env.agent_pos} current state direction {self.env.agent_dir}')
            print(f"Epoch {epoch}: state: {current_state}") #current state somehow wrong 
            # print(f"Epoch {epoch}: Starting state: {current_state}")
            # print(f"Epoch {epoch}: Starting position: {self.state_index_to_position(current_state)}")

            
            
            self.steps = 1
            
            initial_s = np.zeros(self.num_states)
            initial_s[current_state] = 1 # populate 1 only at the index that s is in ( one hot encoding)
            
            while current_state not in goal_states:
                print(f"Before step: {current_state}, Pos: {self.env.agent_pos}, Dir: {self.env.agent_dir}")
                #action = np.argmax(self.Free_energy_table[current_state])
                #fix action choice
                action = np.random.choice(np.arange(self.num_actions), p=self.Pi_a_s[current_state])
                print(f"Step {self.steps}: Action {action} taken.")
                # print(f"greedy action: {action}, exploiting")


                #transition to the next state
                next_obs, _, done, _, _ = self.env.step(action)

                next_state = self.position_to_state_index()
                print(f'next step position {self.env.agent_pos}, direction {self.env.agent_dir}')
                # print(f"next_observation: {next_obs}")
                
                print(f"next_state: {next_state}")
                
                reward = 0 if next_state in goal_states else -1
                
                # print(f"initial_s: {initial_s}")
                self.Ps_s_matrix = np.linalg.matrix_power(self.P_s_by_s, self.steps)  #TODO how should we index so it should be steps minus one?
                # print(f'P_s_by_s_matrix: {self.Ps_s_matrix}')
                # print(f'Ps_by_s_matrix: {self.P_s_by_s}')
                # print(f'nan locations in self.P_s_by_s: {np.argwhere(np.isnan(self.P_s_by_s))}')
                # print(f'P_s_by_s matrix to the power of steps: {Ps_s}')
                self.P_s = np.dot(initial_s .T, self.Ps_s_matrix)   #define P(s)
                assert np.isclose(np.sum(self.P_s), 1), f"Sum of P(s) is not 1: {np.sum(self.P_s)}"
                # print(f'P_s: {self.P_s}')
                # print(f'nan locations ins self.P_s: {np.argwhere(np.isnan(self.P_s))}')
                # print(f'sum of P_s: {np.sum(self.P_s)}')


                # self.Ps_t_plus1_matrix = np.linalg.matrix_power(self.P_s_by_s, self.steps)
                # self.P_s_tplus1 = np.dot(initial_s.T, self.Ps_t_plus1_matrix) #define P(s')
                # assert np.isclose(np.sum(self.P_s_tplus1), 1), f"Sum of P(s') is not 1: {np.sum(self.P_s_tplus1)}"
                # print(f'sum of P_s_tplus1: {np.sum(self.P_s_tplus1)}')
                
                # print(f'P_s: {self.P_s}, P_s_shape: {self.P_s.shape}')
               
                #for self.Pi(a) marginal
                # rows_of_actions_per_state = []
                # for s in range(self.num_states):
                #     row = self.Pi_a_s[s] * self.P_s[s]
                #     # print(f'row in Pi:{self.Pi_a_s[s]} ')
                #     # print(f'row {row}')
                #     rows_of_actions_per_state.append(row)
                    
                self.Pi_a  = np.full(self.num_actions, 1/self.num_actions) #np.sum(np.asarray(rows_of_actions_per_state), axis=0)
                # print(f"Pi_a: {self.Pi_a}")
                sum_of_Pi_a = np.sum(self.Pi_a)
                assert np.isclose(sum_of_Pi_a, 1), f"Sum of Pi(a) is not 1: {sum_of_Pi_a}"
                # print(f"sum of Pi(a) is: {sum_of_Pi_a}")
               
                
                if self.deterministic == False:
                    F_0 = 0
                    for s in range(self.num_states):
                        F_0 += ((np.log(self.P_s_given_s_a[current_state, action, s]) - np.log(self.P_s[s]) - (self.beta * reward)) * self.P_s_given_s_a[current_state, action, s] )
                        G_0 = 0 
                        for a in range(self.actions):
                            G_0 += (np.log(self.Pi_a_s[s, a]) - np.log(self.Pi_a[a]) + self.Free_energy_table[s, a]) * self.Pi_a_s[s, a]
                        F_0+= G_0 * (self.P_s_given_s_a[current_state, action, s])


                if self.deterministic == True:
                    # print(f'P_s_given_s_a at current state and action: {self.P_s_given_s_a[current_state, action]}')
                    # print(f'current state and action from P_s_given_s_a: {current_state, action}')
                   
                    s_t_plus_1 = next_state.copy() # also known as next state
                    
                    print(f"state t plus 1: {s_t_plus_1}") 
                    # print(f'state t plus 1 coordiantes: {self.state_index_to_position(s_t_plus_1)}')
                    # print(f'self.P_s[s_t_plus_1]:{self.P_s[s_t_plus_1]}')
                    F_0 = (-np.log(self.P_s[s_t_plus_1])) - (self.beta * reward)
                    # print(f'F_0: {F_0}')
                    # print(f'np.log(self.P_s[s_t_plus_1]:{np.log(self.P_s[s_t_plus_1])}')
                    # print(f'log of P_s: {np.log(self.P_s)}')
                    G_0 = 0

                    for a in range(self.num_actions):
                        G_0 += (np.log(self.Pi_a_s[s_t_plus_1, a]) - np.log(self.Pi_a[a]) + self.Free_energy_table[s_t_plus_1, a]) * self.Pi_a_s[s_t_plus_1, a]
                        # print(f'G_0: {G_0}')
                    F_0 += G_0 # dont need to multiply by P(s'|s,a) because its deterministic, ==1
                   
                self.Free_energy_table[current_state, action] = F_0
                # print(f'Free energy table: {self.Free_energy_table}')
                
                current_state = s_t_plus_1
                self.steps+=1

                # if self.steps == 3:
                #     break

                #system of equations
                
                

                
                #for self.Z(s, Beta)
                element_wise_a_by_q_table = self.Pi_a * np.exp(-self.Free_energy_table)
                
                # print(f'shape value inside exponential{self.Free_energy_table.shape}')
                # print(f'exponential of Free energy table: {np.exp(-self.Free_energy_table)}')
                # print(f'shape of element wise a by q table: {element_wise_a_by_q_table.shape}')
                self.Zeta = np.sum((element_wise_a_by_q_table), axis = 1)
                # print (f"Zeta: {self.Zeta.shape}")
          
                    
                #for normalizing the policy pi(a|s)
                self.Pi_a_s = (self.Pi_a * np.exp(-self.Free_energy_table)) / self.Zeta.reshape(-1,1)
                # print(f"Pi_a_s, shape: {self.Pi_a_s.shape}")
                # print(f"Pi_a_s: {self.Pi_a_s}")
                if self.Pi_a_s.shape != self.Free_energy_table.shape:
                    raise ValueError (f"Pi_a_s is not equal to the Q table: {self.Pi_a_s.shape}")
                if not np.isclose(np.sum(self.Pi_a_s[0]), 1):
                    print(f"Sum of Pi_a_s is not close to 1: {np.sum(self.Pi_a_s, axis=1)}")
                    break
                # assert(False)
                #recalculate the P_s_s matrix using new Pi_a_s
                # self.P_s_by_s = self.find_connected_states()
                # print(f'new P_s_by_s matrix: {self.P_s_by_s}')
                # print(f'nan locations in P_s_by_s: {np.argwhere(np.isnan(self.P_s_by_s))}')

               
                
               
        
    


    def run_policy_2(self):
        """Run the environment using the learned policy from the Free energy table."""

        self.env.reset()[0]  # Reset environment
        current_state = self.position_to_state_index()  # Convert starting position to index
        done = False
        step_count = 0  # Track steps to prevent infinite loops

        while not done and step_count < 100:  # Prevent infinite loops
            # Debugging: Print current state and Q-values
            print(f"Step {step_count}: State {current_state}")
            print(f"Free energy values: {self.Pi_a_s[current_state]}")
            
            action = np.argmax(self.Pi_a_s[current_state])  # Choose best action
            print(f"Chosen action: {action}")

            # Before taking the step, print agent's current position
            print(f"Before step: Agent Pos: {self.env.agent_pos}, Dir: {self.env.agent_dir}")

            next_obs, _, done, _, _ = self.env.step(action)  # Take action

            # After step, print agent's new position
            print(f"After step: Agent Pos: {self.env.agent_pos}, Dir: {self.env.agent_dir}")

            next_state = self.position_to_state_index()  # Convert new state

            # Detect if the agent is looping in the same state
            if next_state == current_state:
                print(f"⚠️ Warning: Agent is stuck! Current state {current_state} is the same as next state {next_state}.")
                break  # Prevent infinite loop

            self.env.render()  # Visualize movement
            current_state = next_state  # Update current state
            step_count += 1





In [592]:
class blahut:
    def __init__(
        self,
        env,
        epochs = 1000,
        beta = 1,
        deterministic = True
         
    ):
        #eventually need to replace num_states with the number of states in the environment formally
        self.env = env # MiniGrid environment called from the class Minigrid
        self.num_states = ((env.width -2) * (env.height -2) *4)
        self.num_actions = env.action_space.n
        self.deterministic = deterministic
        self.Free_energy_table = np.zeros((self.num_states, self.num_actions)) # Q table has goal states in it as well
        print(f"Free energy table shape: {self.Free_energy_table.shape}")
        self.epochs = epochs
        self.Pi_a = np.full(self.num_actions, 1/self.num_actions)
        self.Pi_a_s = np.full((self.num_states, self.num_actions), 1 / self.num_actions)
        print(f"policy shape: {self.Pi_a_s.shape}")
        self.P_s_by_s = np.zeros((self.num_states, self.num_states)) # helps caluclate P(s'|s), to then calculate P(s)
        print(f"s by s shape: {self.P_s_by_s.shape}")
        self.allowed_state_idx = self.find_state_indexes(env)
        self.steps = 0 #within each epoch, this counts the number of state transitions
        self.Zeta = np.zeros(self.num_states)
        self.P_s = np.zeros(self.num_states)
        self.P_s_tplus1 = np.zeros(self.num_states)
        self.P_s_given_s_a = np.zeros((self.num_states, self.num_actions, self.num_states))
        self.beta = beta
        self.Pi_a_tplus1 = np.zeros(self.num_actions)

        # print(f"allowed state indexes: {self.allowed_state_idx}")


    
    def position_to_state_index(self, tuple_position = None): 
        grid_width = self.env.grid.width -2
        if tuple_position is None:
            direction = self.env.agent_dir
            x, y = self.env.agent_pos

        else:
            if not isinstance(tuple_position, tuple) or len(tuple_position) != 3:
                raise ValueError(f"Invalid position format: {tuple_position}")
            x, y, direction = tuple_position
        return np.int64(((y-1) * grid_width + (x-1)) * 4 + direction) 
    
    def state_index_to_position(self, state_idx):
        """Converts a scalar state index back into (x, y, direction)."""
        grid_width = self.env.grid.width-2
        
        direction = state_idx % 4
        linear_idx = state_idx // 4

        y, x = divmod(linear_idx, grid_width)  # Convert to (x, y)
        
        return x+1, y+1, direction
    
    
    def find_state_indexes(self, env):
        """Counts all states except walls and barriers"""
        state_indexes_list = []
        for x in range(1, env.grid.width-1):
            for y in range(1, env.grid.height-1):
                if env.grid.get(x, y) is None: #grabs all empty spaces
                    for direction in range(4):
                        state_index = self.position_to_state_index((x, y, direction))
                        state_indexes_list.append(state_index)
        return state_indexes_list        
    
    def next_state_index(self, current_state_idx, action):
        # Convert current state index into (x, y, direction)
        x, y, direction = self.state_index_to_position(current_state_idx )
        
        if action == 0:  # Turn left
            direction = (direction - 1) % 4
        elif action == 1:  # Turn right
            direction = (direction + 1) % 4
        elif action == 2:  # Move forward
            if direction == 0:   # looking right increase x
                x += 1
            elif direction == 1: # looking down increase y
                y += 1
            elif direction == 2: # facing left: decrease x
                x -= 1
            elif direction == 3: # looking up: decrease y
                y -= 1
        else:
            raise ValueError(f"Invalid action: {action}")
        
        # Convert the new (x, y, direction) back to a state index.
        next_state = self.position_to_state_index((x, y, direction))
        if next_state not in self.allowed_state_idx:
            # If the next state is not allowed, return the current state
            return current_state_idx
        return next_state
    
    def find_all_next_states(self):
        """Find all possible next states for each state and action."""
        for state in self.allowed_state_idx:
            for action in range(self.num_actions):
                next_state = self.next_state_index(state, action)
                self.P_s_given_s_a[state, action, next_state] = 1
                # print(f"State {state}, Action {action} leads to State {next_state}")

    def find_connected_states(self):
        """
        Build and return a connectivity matrix P_s_by_s where the element at [state, next_state]
        is given by the probability from self.Pi_a_s for the action that leads from state to next_state.
        """
        # Loop over all states.
        for state in self.allowed_state_idx:
            # Loop over all actions for the state.
            for action in range(self.num_actions):
                # Find the next state: assume a deterministic transition where exactly one entry is 1.
                next_state = np.argmax(self.P_s_given_s_a[state, action, :])
                # Set the connectivity matrix: you might choose to sum if multiple actions lead to the same state.
                self.P_s_by_s[state, next_state] = self.Pi_a_s[state, action]
        return self.P_s_by_s
    


    def train(self, epochs):
        """reworked order of equations using the Blahut Arimoto Algorithm WITHOUT CONVERGENCE
        first: initialize policy pi(a|s) to be uniform, based on policy populate P(s|s,a), P(s,s) and P(s), initialize pi(a) 
        1. solve for Free energy table
        2. solve or pi(a)
        3. solve for Z(s)
        4. solve for  new pi(a|s)
           recalculate P(s,s) and P(s) using new pi(a|s)    -------> at this step would run to convergence by repeating 1-4
        5. move on to the next state using the new pi(a|s)
        6. repeat
        """
        
        
        self.find_all_next_states() #generates P(s'|s,a) matrix
        
        goal_states = [self.position_to_state_index((8, 1, d)) for d in range(4)] #goal state index
         
        

        # print(f'P_s_by_s matrix: {self.P_s_by_s}')


        for epoch in range(epochs):

            current_state = np.random.choice(self.allowed_state_idx)
            x, y,dir = self.state_index_to_position(current_state)
        
            self.env.agent_pos = (x,y)
            self.env.agent_dir = dir
            print(f'current state position: {self.env.agent_pos} current state direction {self.env.agent_dir}')
            print(f"Epoch {epoch}: state: {current_state}") #current state somehow wrong 
            # print(f"Epoch {epoch}: Starting state: {current_state}")
            # print(f"Epoch {epoch}: Starting position: {self.state_index_to_position(current_state)}")

            
            
            self.steps = 1
            
            initial_s = np.zeros(self.num_states)
            initial_s[current_state] = 1 # populate 1 only at the index that s is in ( one hot encoding)
            
            while current_state not in goal_states:
                print(f"Before step: {current_state}, Pos: {self.env.agent_pos}, Dir: {self.env.agent_dir}")
                #action = np.argmax(self.Free_energy_table[current_state])
                #fix action choice
                action = np.random.choice(np.arange(self.num_actions), p=self.Pi_a_s[current_state])
                print(f"Step {self.steps}: Action {action} taken.")
                # print(f"greedy action: {action}, exploiting")


                #transition to the next state
                next_obs, _, done, _, _ = self.env.step(action)

                next_state = self.position_to_state_index()
                print(f'next step position {self.env.agent_pos}, direction {self.env.agent_dir}')
                # print(f"next_observation: {next_obs}")
                
                print(f"next_state: {next_state}")
                
                reward = 0 if next_state in goal_states else -1
                
                # print(f"initial_s: {initial_s}")
                
                self.find_connected_states() #call this to initialize the P(s'|s) matrix

                # print(f"initial_s: {initial_s}")
                self.Ps_s_matrix = np.linalg.matrix_power(self.P_s_by_s, self.steps)  #TODO how should we index so it should be steps minus one?
                # print(f'P_s_by_s_matrix: {self.Ps_s_matrix}')
                # print(f'Ps_by_s_matrix: {self.P_s_by_s}')
                # print(f'nan locations in self.P_s_by_s: {np.argwhere(np.isnan(self.P_s_by_s))}')
                # print(f'P_s_by_s matrix to the power of steps: {Ps_s}')
                self.P_s = np.dot(initial_s .T, self.Ps_s_matrix)   #define P(s)
                assert np.isclose(np.sum(self.P_s), 1), f"Sum of P(s) is not 1: {np.sum(self.P_s)}"
                # print(f'P_s: {self.P_s}')
                # print(f'nan locations ins self.P_s: {np.argwhere(np.isnan(self.P_s))}')
                # print(f'sum of P_s: {np.sum(self.P_s)}')
                # # print(f'P_s: {self.P_s}, P_s_shape: {self.P_s.shape}')
               
                if self.deterministic == True:
                    # print(f'P_s_given_s_a at current state and action: {self.P_s_given_s_a[current_state, action]}')
                    # print(f'current state and action from P_s_given_s_a: {current_state, action}')
                   
                    s_t_plus_1 = next_state.copy() # also known as next state
                    
                    print(f"state t plus 1: {s_t_plus_1}") 
                    # print(f'state t plus 1 coordiantes: {self.state_index_to_position(s_t_plus_1)}')
                    # print(f'self.P_s[s_t_plus_1]:{self.P_s[s_t_plus_1]}')
                    F_0 = (-np.log(self.P_s[s_t_plus_1]+ 1e-15) ) - (self.beta * reward)
                    # print(f'F_0: {F_0}')
                    # print(f'np.log(self.P_s[s_t_plus_1]:{np.log(self.P_s[s_t_plus_1])}')
                    # print(f'log of P_s: {np.log(self.P_s)}')
                    G_0 = 0

                    for a in range(self.num_actions):
                        G_0 += (np.log(self.Pi_a_s[s_t_plus_1, a] + 1e-15) - np.log(self.Pi_a[a]+1e-15) + self.Free_energy_table[s_t_plus_1, a]) * self.Pi_a_s[s_t_plus_1, a]
                        # print(f'G_0: {G_0}')
                    F_0 += G_0 # dont need to multiply by P(s'|s,a) because its deterministic, ==1
                    if np.isnan(F_0):
                        print(f"F_0 is NaN at state {current_state} and action {action}")
                        print(f'self.P_s[s_t_plus_1]:{self.P_s[s_t_plus_1]}')
                        print(f"reward: {reward}")
                        for a in range(self.num_actions):
                            print(f'Pi_a_s[s_t_plus_1, a]:{self.Pi_a_s[s_t_plus_1, a]}')
                            print(f'Pi_a[a]:{self.Pi_a[a]}')
                            print(f'Free_energy_table[s_t_plus_1, a]:{self.Free_energy_table[s_t_plus_1, a]}')
                        
                        raise ValueError("F_0 is NaN") 
                    if np.isinf(F_0):
                        print(f"F_0 is inf at state {current_state} and action {action}")
                        print(f'self.P_s[s_t_plus_1]:{self.P_s[s_t_plus_1]}')
                        print(f"reward: {reward}")
                        for a in range(self.num_actions):
                            print(f'Pi_a_s[s_t_plus_1, a]:{self.Pi_a_s[s_t_plus_1, a]}')
                            print(f'Pi_a[a]:{self.Pi_a[a]}')
                            print(f'Free_energy_table[s_t_plus_1, a]:{self.Free_energy_table[s_t_plus_1, a]}')
                        
                        raise ValueError("F_0 is inf") 
                self.Free_energy_table[current_state, action] = np.min([F_0, 100])
                self.Free_energy_table[self.allowed_state_idx] = self.Free_energy_table[self.allowed_state_idx] - np.mean(self.Free_energy_table[self.allowed_state_idx])
            
                # print(f'Free energy table: {self.Free_energy_table}')
                
                
                
                #for self.Pi(a) marginal
                rows_of_actions_per_state = []
                for s in range(self.num_states):
                    row = self.Pi_a_s[s] * self.P_s[s]
                    # print(f'row in Pi:{self.Pi_a_s[s]} ')
                    # print(f'row {row}')
                    rows_of_actions_per_state.append(row)
                    
                self.Pi_a  = np.full(self.num_actions, 1/self.num_actions)#np.sum(np.asarray(rows_of_actions_per_state), axis=0)#   
                # print(f"Pi_a: {self.Pi_a}")
                sum_of_Pi_a = np.sum(self.Pi_a)
                assert np.isclose(sum_of_Pi_a, 1), f"Sum of Pi(a) is not 1: {sum_of_Pi_a}"
                # print(f"sum of Pi(a) is: {sum_of_Pi_a}")
               
                #for self.Z(s, Beta)
                element_wise_a_by_q_table = self.Pi_a * np.exp(-self.Free_energy_table)
                
                # print(f'shape value inside exponential{self.Free_energy_table.shape}')
                # print(f'exponential of Free energy table: {np.exp(-self.Free_energy_table)}')
                # print(f'shape of element wise a by q table: {element_wise_a_by_q_table.shape}')
                self.Zeta = np.sum((element_wise_a_by_q_table), axis = 1)
                # print (f"Zeta: {self.Zeta.shape}")
          
                    
                
                #for normalizing the policy pi(a|s)
                self.Pi_a_s = (self.Pi_a * np.exp(-self.Free_energy_table)) / self.Zeta.reshape(-1,1)
                # print(f"Pi_a_s, shape: {self.Pi_a_s.shape}")
                # print(f"Pi_a_s: {self.Pi_a_s}")
                if self.Pi_a_s.shape != self.Free_energy_table.shape:
                    raise ValueError (f"Pi_a_s is not equal to the Q table: {self.Pi_a_s.shape}")
                if not np.isclose(np.sum(self.Pi_a_s[0]), 1):
                    print(f"Sum of Pi_a_s is not close to 1: {np.sum(self.Pi_a_s, axis=1)}")
                    break
     
                #recalculate the P_s_s matrix using new Pi_a_s
                # self.P_s_by_s = self.find_connected_states()
                # print(f'new P_s_by_s matrix: {self.P_s_by_s}')
                # print(f'nan locations in P_s_by_s: {np.argwhere(np.isnan(self.P_s_by_s))}')
                current_state = s_t_plus_1
                self.steps+=1

                # assert(False)
               
                
               
        
    


    def run_policy_2(self):
        """Run the environment using the learned policy from the Free energy table."""

        self.env.reset()[0]  # Reset environment
        current_state = self.position_to_state_index()  # Convert starting position to index
        done = False
        step_count = 0  # Track steps to prevent infinite loops

        while not done and step_count < 100:  # Prevent infinite loops
            # Debugging: Print current state and Q-values
            print(f"Step {step_count}: State {current_state}")
            print(f"Free energy values: {self.Pi_a_s[current_state]}")
            
            action = np.random.choice(np.arange(self.num_actions), p=self.Pi_a_s[current_state]) #np.argmax(self.Pi_a_s[current_state])  # Choose best action
            print(f"Chosen action: {action}")

            # Before taking the step, print agent's current position
            print(f"Before step: Agent Pos: {self.env.agent_pos}, Dir: {self.env.agent_dir}")

            next_obs, _, done, _, _ = self.env.step(action)  # Take action

            # After step, print agent's new position
            print(f"After step: Agent Pos: {self.env.agent_pos}, Dir: {self.env.agent_dir}")

            next_state = self.position_to_state_index()  # Convert new state

            # Detect if the agent is looping in the same state
            # if next_state == current_state:
            #     print(f"⚠️ Warning: Agent is stuck! Current state {current_state} is the same as next state {next_state}.")
            #     break  # Prevent infinite loop

            self.env.render()  # Visualize movement
            current_state = next_state  # Update current state
            step_count += 1





In [9]:
class blahut_witerations:
    def __init__(
        self,
        env,
        epochs = 1000,
        beta = 5,
        deterministic = True
         
    ):
        #eventually need to replace num_states with the number of states in the environment formally
        self.env = env # MiniGrid environment called from the class Minigrid
        self.num_states = ((env.width -2) * (env.height -2) *4)
        self.num_actions = env.action_space.n
        self.deterministic = deterministic
        self.Free_energy_table = np.zeros((self.num_states, self.num_actions)) # Q table has goal states in it as well
        print(f"Free energy table shape: {self.Free_energy_table.shape}")
        self.epochs = epochs
        self.Pi_a = np.full(self.num_actions, 1/self.num_actions)
        self.Pi_a_s = np.full((self.num_states, self.num_actions), 1 / self.num_actions)
        print(f"policy shape: {self.Pi_a_s.shape}")
        self.P_s_by_s = np.zeros((self.num_states, self.num_states)) # helps caluclate P(s'|s), to then calculate P(s)
        print(f"s by s shape: {self.P_s_by_s.shape}")
        self.allowed_state_idx = self.find_state_indexes(env)
        self.steps = 0 #within each epoch, this counts the number of state transitions
        self.Zeta = np.zeros(self.num_states)
        self.P_s = np.zeros(self.num_states)
        self.P_s_tplus1 = np.zeros(self.num_states)
        self.P_s_given_s_a = np.zeros((self.num_states, self.num_actions, self.num_states))
        self.beta = beta
        self.Pi_a_tplus1 = np.zeros(self.num_actions)

        # print(f"allowed state indexes: {self.allowed_state_idx}")


    
    def position_to_state_index(self, tuple_position = None): 
        grid_width = self.env.grid.width -2
        if tuple_position is None:
            direction = self.env.agent_dir
            x, y = self.env.agent_pos

        else:
            if not isinstance(tuple_position, tuple) or len(tuple_position) != 3:
                raise ValueError(f"Invalid position format: {tuple_position}")
            x, y, direction = tuple_position
        return np.int64(((y-1) * grid_width + (x-1)) * 4 + direction) 
    
    def state_index_to_position(self, state_idx):
        """Converts a scalar state index back into (x, y, direction)."""
        grid_width = self.env.grid.width-2
        
        direction = state_idx % 4
        linear_idx = state_idx // 4

        y, x = divmod(linear_idx, grid_width)  # Convert to (x, y)
        
        return x+1, y+1, direction
    
    
    def find_state_indexes(self, env):
        """Counts all states except walls and barriers"""
        state_indexes_list = []
        for x in range(1, env.grid.width-1):
            for y in range(1, env.grid.height-1):
                if env.grid.get(x, y) is None: #grabs all empty spaces
                    for direction in range(4):
                        state_index = self.position_to_state_index((x, y, direction))
                        state_indexes_list.append(state_index)
        return state_indexes_list        
    
    def next_state_index(self, current_state_idx, action):
        # Convert current state index into (x, y, direction)
        x, y, direction = self.state_index_to_position(current_state_idx )
        
        if action == 0:  # Turn left
            direction = (direction - 1) % 4
        elif action == 1:  # Turn right
            direction = (direction + 1) % 4
        elif action == 2:  # Move forward
            if direction == 0:   # looking right increase x
                x += 1
            elif direction == 1: # looking down increase y
                y += 1
            elif direction == 2: # facing left: decrease x
                x -= 1
            elif direction == 3: # looking up: decrease y
                y -= 1
        else:
            raise ValueError(f"Invalid action: {action}")
        
        # Convert the new (x, y, direction) back to a state index.
        next_state = self.position_to_state_index((x, y, direction))
        if next_state not in self.allowed_state_idx:
            # If the next state is not allowed, return the current state
            return current_state_idx
        return next_state
    
    def find_all_next_states(self):
        """Find all possible next states for each state and action."""
        for state in self.allowed_state_idx:
            for action in range(self.num_actions):
                next_state = self.next_state_index(state, action)
                self.P_s_given_s_a[state, action, next_state] = 1
                # print(f"State {state}, Action {action} leads to State {next_state}")

    def find_connected_states(self):
        """
        Build and return a connectivity matrix P_s_by_s where the element at [state, next_state]
        is given by the probability from self.Pi_a_s for the action that leads from state to next_state.
        """
        # Reset the connectivity matrix at the beginning.
        
        # Loop over all states.
        for state in self.allowed_state_idx:
            # Loop over all actions for the state.
            for action in range(self.num_actions):
                # Find the next state: assume a deterministic transition where exactly one entry is 1.
                next_state = np.argmax(self.P_s_given_s_a[state, action, :])
                # Set the connectivity matrix: you might choose to sum if multiple actions lead to the same state.
                self.P_s_by_s[state, next_state] = self.Pi_a_s[state, action]
        return self.P_s_by_s
    


    def train(self):
        """reworked order of equations using the Blahut Arimoto Algorithm WITH CONVERGENCE
        first: initialize policy pi(a|s) to be uniform, based on policy populate P(s|s,a), P(s,s) and P(s), initialize pi(a) 
        1. solve for Free energy table
        2. solve or pi(a)
        3. solve for Z(s)
        4. solve for  new pi(a|s)
           recalculate P(s,s) and P(s) using new pi(a|s)    -------> at this step would run to convergence by repeating 1-4
        5. move on to the next state using the new pi(a|s)
        6. repeat
        """
        max_steps = 400 #max number of steps to take in the environment
        tol = 1e-2
        max_inner_iters = 100
        

        
        self.find_all_next_states() #generates P(s'|s,a) matrix
        
        goal_states = [self.position_to_state_index((8, 1, d)) for d in range(4)] #goal state index

        self.steps = 1 
        #update the 

        current_state = np.random.choice(self.allowed_state_idx)
        x, y,dir = self.state_index_to_position(current_state)
    
        self.env.agent_pos = (x,y)
        self.env.agent_dir = dir
        print(f'current state position: {self.env.agent_pos} current state direction {self.env.agent_dir}, state index {current_state}')
        initial_s = np.zeros(self.num_states)
        initial_s[current_state] = 1 
        self.free_energy_info = []
    
        while current_state not in goal_states:
            action = np.random.choice(np.arange(self.num_actions), p=self.Pi_a_s[current_state])
            print(f"Step {self.steps}: Action {action} taken.")
            # print(f"greedy action: {action}, exploiting")

            #transition to the next state
            next_obs, _, done, _, _ = self.env.step(action)

            next_state = self.position_to_state_index()
            print(f'next step position {self.env.agent_pos}, direction {self.env.agent_dir}')
            print(f"next_state: {next_state}")

            # populate 1 only at the index that s is in ( one hot encoding)
            

            reward = 0 if next_state in goal_states else -1

            iteration_count = 0
            
            while iteration_count < max_inner_iters:
                old_Pi_a_s = self.Pi_a_s.copy()
                
                
                
                # print(f"initial_s: {initial_s}")
                
                self.find_connected_states() #call this to initialize the P(s'|s) matrix

                # print(f"initial_s: {initial_s}")
                self.Ps_s_matrix = np.linalg.matrix_power(self.P_s_by_s, self.steps)  #TODO how should we index so it should be steps minus one?
                # print(f'P_s_by_s_matrix: {self.Ps_s_matrix}')
                # print(f'Ps_by_s_matrix: {self.P_s_by_s}')
                # print(f'nan locations in self.P_s_by_s: {np.argwhere(np.isnan(self.P_s_by_s))}')
                # print(f'P_s_by_s matrix to the power of steps: {Ps_s}')
                self.P_s = np.dot(initial_s .T, self.Ps_s_matrix)   #define P(s)
                assert np.isclose(np.sum(self.P_s), 1), f"Sum of P(s) is not 1: {np.sum(self.P_s)}"
                # print(f'P_s: {self.P_s}')
                # print(f'nan locations ins self.P_s: {np.argwhere(np.isnan(self.P_s))}')
                # print(f'sum of P_s: {np.sum(self.P_s)}')
                # # print(f'P_s: {self.P_s}, P_s_shape: {self.P_s.shape}')
            
                if self.deterministic == True:
                    # print(f'P_s_given_s_a at current state and action: {self.P_s_given_s_a[current_state, action]}')
                    # print(f'current state and action from P_s_given_s_a: {current_state, action}')
                
                    s_t_plus_1 = next_state.copy() # also known as next state
                    
                    print(f"state t plus 1: {s_t_plus_1}") 
                    # print(f'state t plus 1 coordiantes: {self.state_index_to_position(s_t_plus_1)}')
                    # print(f'self.P_s[s_t_plus_1]:{self.P_s[s_t_plus_1]}')
                    F_0 = (-np.log(self.P_s[s_t_plus_1]+ 1e-15) ) - (self.beta * reward)
                    # print(f'F_0: {F_0}')
                    # print(f'np.log(self.P_s[s_t_plus_1]:{np.log(self.P_s[s_t_plus_1])}')
                    # print(f'log of P_s: {np.log(self.P_s)}')
                    G_0 = 0

                    for a in range(self.num_actions):
                        G_0 += (np.log(self.Pi_a_s[s_t_plus_1, a] + 1e-15) - np.log(self.Pi_a[a]+1e-15) + self.Free_energy_table[s_t_plus_1, a]) * self.Pi_a_s[s_t_plus_1, a]
                        # print(f'G_0: {G_0}')
                    F_0 += G_0 # dont need to multiply by P(s'|s,a) because its deterministic, ==1
                    if np.isnan(F_0):
                        print(f"F_0 is NaN at state {current_state} and action {action}")
                        print(f'self.P_s[s_t_plus_1]:{self.P_s[s_t_plus_1]}')
                        print(f"reward: {reward}")
                        for a in range(self.num_actions):
                            print(f'Pi_a_s[s_t_plus_1, a]:{self.Pi_a_s[s_t_plus_1, a]}')
                            print(f'Pi_a[a]:{self.Pi_a[a]}')
                            print(f'Free_energy_table[s_t_plus_1, a]:{self.Free_energy_table[s_t_plus_1, a]}')
                        
                        raise ValueError("F_0 is NaN") 
                    if np.isinf(F_0):
                        print(f"F_0 is inf at state {current_state} and action {action}")
                        print(f'self.P_s[s_t_plus_1]:{self.P_s[s_t_plus_1]}')
                        print(f"reward: {reward}")
                        for a in range(self.num_actions):
                            print(f'Pi_a_s[s_t_plus_1, a]:{self.Pi_a_s[s_t_plus_1, a]}')
                            print(f'Pi_a[a]:{self.Pi_a[a]}')
                            print(f'Free_energy_table[s_t_plus_1, a]:{self.Free_energy_table[s_t_plus_1, a]}')
                        
                        raise ValueError("F_0 is inf") 
                self.Free_energy_table[current_state, action] = np.min([F_0, 100])
                self.Free_energy_table[self.allowed_state_idx] = self.Free_energy_table[self.allowed_state_idx] - np.mean(self.Free_energy_table[self.allowed_state_idx])
                # print(f'Free energy table: {self.Free_energy_table}')
                
                #for self.Pi(a) marginal
                rows_of_actions_per_state = []
                for s in range(self.num_states):
                    row = self.Pi_a_s[s] * self.P_s[s]
                    # print(f'row in Pi:{self.Pi_a_s[s]} ')
                    # print(f'row {row}')
                    rows_of_actions_per_state.append(row)
                    
                self.Pi_a  = np.sum(np.asarray(rows_of_actions_per_state), axis=0) #np.full(self.num_actions, 1/self.num_actions) # #   ##  
                # print(f"Pi_a: {self.Pi_a}")
                old_Pi_a = self.Pi_a.copy()
                sum_of_Pi_a = np.sum(self.Pi_a)
                assert np.isclose(sum_of_Pi_a, 1), f"Sum of Pi(a) is not 1: {sum_of_Pi_a}"
                # print(f"sum of Pi(a) is: {sum_of_Pi_a}")
            
                #for self.Z(s, Beta)
                element_wise_a_by_q_table = self.Pi_a * np.exp(-self.Free_energy_table)
                
                # print(f'shape value inside exponential{self.Free_energy_table.shape}')
                # print(f'exponential of Free energy table: {np.exp(-self.Free_energy_table)}')
                # print(f'shape of element wise a by q table: {element_wise_a_by_q_table.shape}')
                self.Zeta = np.sum((element_wise_a_by_q_table), axis = 1)
                # print (f"Zeta: {self.Zeta.shape}")
        
                    
                
                #for normalizing the policy pi(a|s)
                self.Pi_a_s = (self.Pi_a * np.exp(-self.Free_energy_table)) / self.Zeta.reshape(-1,1)
                # print(f"Pi_a_s, shape: {self.Pi_a_s.shape}")
                # print(f"Pi_a_s: {self.Pi_a_s}")
                if self.Pi_a_s.shape != self.Free_energy_table.shape:
                    raise ValueError (f"Pi_a_s is not equal to the Q table: {self.Pi_a_s.shape}")
                if not np.isclose(np.sum(self.Pi_a_s[0]), 1):
                    print(f"Sum of Pi_a_s is not close to 1: {np.sum(self.Pi_a_s, axis=1)}")
                    break
    
                
                diff_pias = np.linalg.norm(self.Pi_a_s - old_Pi_a_s) 
                diff_pia = np.linalg.norm(self.Pi_a - old_Pi_a)
                
                if diff_pias < tol : #trained till convergence good to move on to the next step
                    print(f"Convergence reached with diff_pias={diff_pias}")
                    self.free_energy_info.append((self.steps, self.Free_energy_table[current_state], current_state, action, self.Pi_a_s[current_state]))
                    break

                else:
                        iteration_count += 1 #need to update the P(s'|s) matrix since the policy changes iteration by iteration
                    
            current_state = next_state
            
            if self.steps == max_steps:
                break
            else:
                self.steps += 1
            


        # assert(False)
            
            
            
    



    def run_policy_2(self):
        """Run the environment using the learned policy from the Free energy table."""

        self.env.reset()[0]  # Reset environment
        current_state = self.position_to_state_index()  # Convert starting position to index
        done = False
        step_count = 0  # Track steps to prevent infinite loops

        while not done and step_count < 100:  # Prevent infinite loops
            # Debugging: Print current state and Q-values
            print(f"Step {step_count}: State {current_state}")
            print(f"Free energy values: {self.Pi_a_s[current_state]}")
            
            action = np.random.choice(np.arange(self.num_actions), p=self.Pi_a_s[current_state]) #np.argmax(self.Pi_a_s[current_state])  # Choose best action
            print(f"Chosen action: {action}")

            # Before taking the step, print agent's current position
            print(f"Before step: Agent Pos: {self.env.agent_pos}, Dir: {self.env.agent_dir}")

            next_obs, _, done, _, _ = self.env.step(action)  # Take action

            # After step, print agent's new position
            print(f"After step: Agent Pos: {self.env.agent_pos}, Dir: {self.env.agent_dir}")

            next_state = self.position_to_state_index()  # Convert new state

            # Detect if the agent is looping in the same state
            # if next_state == current_state:
            #     print(f"⚠️ Warning: Agent is stuck! Current state {current_state} is the same as next state {next_state}.")
            #     break  # Prevent infinite loop

            self.env.render()  # Visualize movement
            current_state = next_state  # Update current state
            step_count += 1





In [14]:
#print out the values of the info to go and plot them for trouble shooting, fixed policy 2 code 


class blahut_w_printouts:
    def __init__(
        self,
        env,
        epochs = 1000,
        beta = 0.1,
        deterministic = True
         
    ):
        #eventually need to replace num_states with the number of states in the environment formally
        self.env = env # MiniGrid environment called from the class Minigrid
        self.num_states = ((env.width -2) * (env.height -2) *4)
        self.num_actions = env.action_space.n
        self.deterministic = deterministic
        self.Free_energy_table = np.zeros((self.num_states, self.num_actions)) # Q table has goal states in it as well
        print(f"Free energy table shape: {self.Free_energy_table.shape}")
        self.epochs = epochs
        self.Pi_a = np.full(self.num_actions, 1/self.num_actions)
        self.Pi_a_s = np.full((self.num_states, self.num_actions), 1 / self.num_actions)
        print(f"policy shape: {self.Pi_a_s.shape}")
        self.P_s_by_s = np.zeros((self.num_states, self.num_states)) # helps caluclate P(s'|s), to then calculate P(s)
        print(f"s by s shape: {self.P_s_by_s.shape}")
        self.allowed_state_idx = self.find_state_indexes(env)
        self.steps = 0 #within each epoch, this counts the number of state transitions
        self.Zeta = np.zeros(self.num_states)
        self.P_s = np.zeros(self.num_states)
        self.P_s_tplus1 = np.zeros(self.num_states)
        self.P_s_given_s_a = np.zeros((self.num_states, self.num_actions, self.num_states))
        self.beta = beta
        self.Pi_a_tplus1 = np.zeros(self.num_actions)

        # print(f"allowed state indexes: {self.allowed_state_idx}")


    
    def position_to_state_index(self, tuple_position = None): 
        grid_width = self.env.grid.width -2
        if tuple_position is None:
            direction = self.env.agent_dir
            x, y = self.env.agent_pos

        else:
            if not isinstance(tuple_position, tuple) or len(tuple_position) != 3:
                raise ValueError(f"Invalid position format: {tuple_position}")
            x, y, direction = tuple_position
        return np.int64(((y-1) * grid_width + (x-1)) * 4 + direction) 
    
    def state_index_to_position(self, state_idx):
        """Converts a scalar state index back into (x, y, direction)."""
        grid_width = self.env.grid.width-2
        
        direction = state_idx % 4
        linear_idx = state_idx // 4

        y, x = divmod(linear_idx, grid_width)  # Convert to (x, y)
        
        return x+1, y+1, direction
    
    
    def find_state_indexes(self, env):
        """Counts all states except walls and barriers"""
        state_indexes_list = []
        for x in range(1, env.grid.width-1):
            for y in range(1, env.grid.height-1):
                if env.grid.get(x, y) is None: #grabs all empty spaces
                    for direction in range(4):
                        state_index = self.position_to_state_index((x, y, direction))
                        state_indexes_list.append(state_index)
        return state_indexes_list        
    
    def next_state_index(self, current_state_idx, action):
        # Convert current state index into (x, y, direction)
        x, y, direction = self.state_index_to_position(current_state_idx )
        
        if action == 0:  # Turn left
            direction = (direction - 1) % 4
        elif action == 1:  # Turn right
            direction = (direction + 1) % 4
        elif action == 2:  # Move forward
            if direction == 0:   # looking right increase x
                x += 1
            elif direction == 1: # looking down increase y
                y += 1
            elif direction == 2: # facing left: decrease x
                x -= 1
            elif direction == 3: # looking up: decrease y
                y -= 1
        else:
            raise ValueError(f"Invalid action: {action}")
        
        # Convert the new (x, y, direction) back to a state index.
        next_state = self.position_to_state_index((x, y, direction))
        if next_state not in self.allowed_state_idx:
            # If the next state is not allowed, return the current state
            return current_state_idx
        return next_state
    
    def find_all_next_states(self):
        """Find all possible next states for each state and action."""
        for state in self.allowed_state_idx:
            for action in range(self.num_actions):
                next_state = self.next_state_index(state, action)
                self.P_s_given_s_a[state, action, next_state] = 1
                # print(f"State {state}, Action {action} leads to State {next_state}")

    def find_connected_states(self):
        """
        Build and return a connectivity matrix P_s_by_s where the element at [state, next_state]
        is given by the probability from self.Pi_a_s for the action that leads from state to next_state.
        """
        
        # Loop over all states.
        for state in self.allowed_state_idx:
            # Loop over all actions for the state.
            for action in range(self.num_actions):
                # Find the next state: assume a deterministic transition where exactly one entry is 1.
                next_state = np.argmax(self.P_s_given_s_a[state, action, :])
                # Set the connectivity matrix: you might choose to sum if multiple actions lead to the same state.
                self.P_s_by_s[state, next_state] += self.Pi_a_s[state, action]
        return self.P_s_by_s
    


    def train(self):
        """reworked order of equations using the Blahut Arimoto Algorithm WITH CONVERGENCE
        first: initialize policy pi(a|s) to be uniform, based on policy populate P(s|s,a), P(s,s) and P(s), initialize pi(a) 
        1. solve for Free energy table
        2. solve or pi(a)
        3. solve for Z(s)
        4. solve for  new pi(a|s)
           recalculate P(s,s) and P(s) using new pi(a|s)    -------> at this step would run to convergence by repeating 1-4
        5. move on to the next state using the new pi(a|s)
        6. repeat
        """
        max_steps = 400 #max number of steps to take in the environment
        tol = 1e-2
        max_inner_iters = 100
        
        #collect terms
        self.info_to_go_term = []
        self.pi_analysis_term = []

        
        self.find_all_next_states() #generates P(s'|s,a) matrix
        
        goal_states = [self.position_to_state_index((8, 1, d)) for d in range(4)] #goal state index

        self.steps = 1 
        #update the 

        current_state = np.random.choice(self.allowed_state_idx)
        x, y,dir = self.state_index_to_position(current_state)
    
        self.env.agent_pos = (x,y)
        self.env.agent_dir = dir
        print(f'current state position: {self.env.agent_pos} current state direction {self.env.agent_dir}, state index {current_state}')
        
        self.free_energy_info = []
    
        while current_state not in goal_states:
            action = np.random.choice(np.arange(self.num_actions), p=self.Pi_a_s[current_state])
            print(f"Step {self.steps}: Action {action} taken.")
            # print(f"greedy action: {action}, exploiting")

            #transition to the next state
            next_obs, _, done, _, _ = self.env.step(action)

            next_state = self.position_to_state_index()
            print(f'next step position {self.env.agent_pos}, direction {self.env.agent_dir}')
            print(f"next_state: {next_state}")

            # populate 1 only at the index that s is in ( one hot encoding)
            initial_s = np.zeros(self.num_states)
            initial_s[current_state] = 1 

            reward = 0 if next_state in goal_states else -1

            iteration_count = 0
            
            while iteration_count < max_inner_iters:
                old_Pi_a_s = self.Pi_a_s.copy()
                
                
                
                # print(f"initial_s: {initial_s}")
                
                self.find_connected_states() #call this to initialize the P(s'|s) matrix

                # print(f"initial_s: {initial_s}")
                self.Ps_s_matrix = np.linalg.matrix_power(self.P_s_by_s, self.steps)  #TODO how should we index so it should be steps minus one?
                # print(f'P_s_by_s_matrix: {self.Ps_s_matrix}')
                # print(f'Ps_by_s_matrix: {self.P_s_by_s}')
                # print(f'nan locations in self.P_s_by_s: {np.argwhere(np.isnan(self.P_s_by_s))}')
                # print(f'P_s_by_s matrix to the power of steps: {Ps_s}')
                self.P_s = np.dot(initial_s .T, self.Ps_s_matrix)   #define P(s)
                assert np.isclose(np.sum(self.P_s), 1), f"Sum of P(s) is not 1: {np.sum(self.P_s)}"
                # print(f'P_s: {self.P_s}')
                # print(f'nan locations ins self.P_s: {np.argwhere(np.isnan(self.P_s))}')
                # print(f'sum of P_s: {np.sum(self.P_s)}')
                # # print(f'P_s: {self.P_s}, P_s_shape: {self.P_s.shape}')
            
                if self.deterministic == True:
                    # print(f'P_s_given_s_a at current state and action: {self.P_s_given_s_a[current_state, action]}')
                    # print(f'current state and action from P_s_given_s_a: {current_state, action}')
                
                    s_t_plus_1 = next_state.copy() # also known as next state
                    
                    print(f"state t plus 1: {s_t_plus_1}") 
                    # print(f'state t plus 1 coordiantes: {self.state_index_to_position(s_t_plus_1)}')
                    # print(f'self.P_s[s_t_plus_1]:{self.P_s[s_t_plus_1]}')
                    F_0 = (-np.log(self.P_s[s_t_plus_1]+ 1e-15) ) - (self.beta * reward)
                    # print(f'F_0: {F_0}')
                    # print(f'np.log(self.P_s[s_t_plus_1]:{np.log(self.P_s[s_t_plus_1])}')
                    # print(f'log of P_s: {np.log(self.P_s)}')
                    G_0 = 0

                    for a in range(self.num_actions):
                        G_0 += (np.log(self.Pi_a_s[s_t_plus_1, a] + 1e-15) - np.log(self.Pi_a[a]+1e-15) + self.Free_energy_table[s_t_plus_1, a]) * self.Pi_a_s[s_t_plus_1, a]
                        # print(f'G_0: {G_0}')
                    F_0 += G_0 # dont need to multiply by P(s'|s,a) because its deterministic, ==1
                    if np.isnan(F_0):
                        print(f"F_0 is NaN at state {current_state} and action {action}")
                        print(f'self.P_s[s_t_plus_1]:{self.P_s[s_t_plus_1]}')
                        print(f"reward: {reward}")
                        for a in range(self.num_actions):
                            print(f'Pi_a_s[s_t_plus_1, a]:{self.Pi_a_s[s_t_plus_1, a]}')
                            print(f'Pi_a[a]:{self.Pi_a[a]}')
                            print(f'Free_energy_table[s_t_plus_1, a]:{self.Free_energy_table[s_t_plus_1, a]}')
                        
                        raise ValueError("F_0 is NaN") 
                    if np.isinf(F_0):
                        print(f"F_0 is inf at state {current_state} and action {action}")
                        print(f'self.P_s[s_t_plus_1]:{self.P_s[s_t_plus_1]}')
                        print(f"reward: {reward}")
                        for a in range(self.num_actions):
                            print(f'Pi_a_s[s_t_plus_1, a]:{self.Pi_a_s[s_t_plus_1, a]}')
                            print(f'Pi_a[a]:{self.Pi_a[a]}')
                            print(f'Free_energy_table[s_t_plus_1, a]:{self.Free_energy_table[s_t_plus_1, a]}')
                        
                        raise ValueError("F_0 is inf") 
                self.Free_energy_table[current_state, action] = np.min([F_0, 100])
                self.Free_energy_table[self.allowed_state_idx] = self.Free_energy_table[self.allowed_state_idx] - np.mean(self.Free_energy_table[self.allowed_state_idx])
                # print(f'Free energy table: {self.Free_energy_table}')
                
                #for self.Pi(a) marginal
                rows_of_actions_per_state = []
                for s in range(self.num_states):
                    row = self.Pi_a_s[s] * self.P_s[s]
                    # print(f'row in Pi:{self.Pi_a_s[s]} ')
                    # print(f'row {row}')
                    rows_of_actions_per_state.append(row)
                    
                self.Pi_a  = np.sum(np.asarray(rows_of_actions_per_state), axis=0) #np.full(self.num_actions, 1/self.num_actions) # #   ##  
                # print(f"Pi_a: {self.Pi_a}")
                old_Pi_a = self.Pi_a.copy()
                sum_of_Pi_a = np.sum(self.Pi_a)
                assert np.isclose(sum_of_Pi_a, 1), f"Sum of Pi(a) is not 1: {sum_of_Pi_a}"
                # print(f"sum of Pi(a) is: {sum_of_Pi_a}")
            
                #for self.Z(s, Beta)
                element_wise_a_by_q_table = self.Pi_a * np.exp(-self.Free_energy_table)
                
                # print(f'shape value inside exponential{self.Free_energy_table.shape}')
                # print(f'exponential of Free energy table: {np.exp(-self.Free_energy_table)}')
                # print(f'shape of element wise a by q table: {element_wise_a_by_q_table.shape}')
                self.Zeta = np.sum((element_wise_a_by_q_table), axis = 1)
                # print (f"Zeta: {self.Zeta.shape}")
        
                    
                
                #for normalizing the policy pi(a|s)
                self.Pi_a_s = (self.Pi_a * np.exp(-self.Free_energy_table)) / self.Zeta.reshape(-1,1)
                # print(f"Pi_a_s, shape: {self.Pi_a_s.shape}")
                # print(f"Pi_a_s: {self.Pi_a_s}")
                if self.Pi_a_s.shape != self.Free_energy_table.shape:
                    raise ValueError (f"Pi_a_s is not equal to the Q table: {self.Pi_a_s.shape}")
                if not np.isclose(np.sum(self.Pi_a_s[0]), 1):
                    print(f"Sum of Pi_a_s is not close to 1: {np.sum(self.Pi_a_s, axis=1)}")
                    break
                

                #caluculate the terms
                for a in range(self.num_actions):
                    pi_analysis = (np.log(self.Pi_a_s[next_state, a] + 1e-15) - np.log(self.Pi_a[a]+1e-15))
                self.info_to_go_term.append(pi_analysis)
                
                diff_pias = np.linalg.norm(self.Pi_a_s - old_Pi_a_s) 
                diff_pia = np.linalg.norm(self.Pi_a - old_Pi_a)
                
                if diff_pias < tol : #trained till convergence good to move on to the next step
                    print(f"Convergence reached with diff_pias={diff_pias}")
                    self.free_energy_info.append((self.steps, self.Free_energy_table[current_state], current_state, action, self.Pi_a_s[current_state]))
                    break

                else:
                        iteration_count += 1 #need to update the P(s'|s) matrix since the policy changes iteration by iteration
                    
            current_state = next_state
            if self.steps == max_steps:
                break
            else:
                self.steps += 1
            


        # assert(False)
            
            
            
    



    def run_policy_2(self):
        """Run the environment using the learned policy from the Free energy table."""

        self.env.reset()[0]  # Reset environment
        current_state = self.position_to_state_index()  # Convert starting position to index
        done = False
        step_count = 0  # Track steps to prevent infinite loops

        while not done and step_count < 100:  # Prevent infinite loops
            # Debugging: Print current state and Q-values
            print(f"Step {step_count}: State {current_state}")
            print(f"Free energy values: {self.Pi_a_s[current_state]}")
            
            action = np.random.choice(np.arange(self.num_actions), p=self.Pi_a_s[current_state]) #np.argmax(self.Pi_a_s[current_state])  # Choose best action
            print(f"Chosen action: {action}")

            # Before taking the step, print agent's current position
            print(f"Before step: Agent Pos: {self.env.agent_pos}, Dir: {self.env.agent_dir}")

            next_obs, _, done, _, _ = self.env.step(action)  # Take action

            # After step, print agent's new position
            print(f"After step: Agent Pos: {self.env.agent_pos}, Dir: {self.env.agent_dir}")

            next_state = self.position_to_state_index()  # Convert new state

            # Detect if the agent is looping in the same state
            # if next_state == current_state:
            #     print(f"⚠️ Warning: Agent is stuck! Current state {current_state} is the same as next state {next_state}.")
            #     break  # Prevent infinite loop

            self.env.render()  # Visualize movement
            current_state = next_state  # Update current state
            step_count += 1





In [ ]:
#currently testing, this uses a trained pi
class Sandbox(GridMixin):
    def __init__(self, env, state: ModelState):
        self.state = state
        self.env = env
        state: ModelState
        learning_rate = 0.9
        discount_factor = 0.9
        epochs = 50
        beta = 5
        self.epochs = epochs
        self.state = state
        self.state.allowed_state_idx = self.find_state_indexes(env)
        self.state.num_states = ((env.width -2) * (env.height -2) *4)
        self.state.num_actions = env.action_space.n
        #shapes defined for first time
        self.state.P_s_given_s_a = np.zeros((self.state.num_states, self.state.num_actions, self.state.num_states)) # P(s'|s,a) matrix
        self.state.P_s_by_s = np.zeros((self.state.num_states, self.state.num_states)) # P(s'|s) matrix
        self.state.Pi_a_s = np.full((self.state.num_states, self.state.num_actions), 1/self.state.num_actions) # pi(s,a) matrix

        self.Q_table = np.zeros((((env.width -2) * (env.height -2) *4), self.state.num_actions)) # Q table has goal states in it as well
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor # discount factor
        
        self.state.beta = beta

        
        self.Pi_a = np.zeros(self.state.num_actions)
        self.P_s = np.zeros(self.state.num_states)
        
       
        
        
        
        
    """ Perform training using BA updates on the Probabilistic Q Learning System """


    def train_BA(self):
        max_ba_iters = 5 #max iterations for the Blahut-Arimoto update
        self.find_all_next_states() #generates self.P(s'|s,a) matrix
        
        goal_states = [self.position_to_state_index((8, 1, d)) for d in range(4)] #goal state index
        

        for epoch in range(self.epochs):
            self.steps = 1
            current_state = np.random.choice(self.state.allowed_state_idx)
            x,y,dir = self.state_index_to_position(current_state)
            self.env.agent_pos = (x,y)
            self.env.agent_dir = dir

            initial_s = np.zeros(self.state.num_states)
            initial_s[current_state] = 1

            while current_state not in goal_states:
                #get the action using pi
                action = np.random.choice(np.arange(self.state.num_actions), p = self.state.Pi_a_s[current_state])
                #transition to the next state
                next_obs, _, done, _, _ = self.env.step(action)

                # print(f"next_observation: {next_obs}")
                next_state = self.position_to_state_index()

                reward = 0 if next_state in goal_states else -1

                self.Q_table[current_state, action] += self.learning_rate * (reward + self.discount_factor * np.max(self.Q_table[next_state]) - self.Q_table[current_state, action])



                #calculate the policy from Q table
                self.state.Pi_a_s = self.calculate_pi()

                self.find_connected_states() #P_s fixed by old Pi_a_s
                Ps_s_matrix = np.linalg.matrix_power(self.state.P_s_by_s, self.steps)
                P_s = np.dot(initial_s.T, Ps_s_matrix) #define P(s)
                assert np.isclose(np.sum(P_s), 1), f"Sum of P(s) is not 1: {np.sum(P_s)}"
                #######loop of calculations#####
                for _ in range(max_ba_iters):
                    # self.find_connected_states() #P_s fixed by old Pi_a_s
                    # Ps_s_matrix = np.linalg.matrix_power(self.state.P_s_by_s, self.steps)
                    # P_s = np.dot(initial_s.T, Ps_s_matrix) #define P(s)
                    # assert np.isclose(np.sum(P_s), 1), f"Sum of P(s) is not 1: {np.sum(P_s)}"
                       
                    
                    # pi_a_sums = []
                    # for s in range(self.state.num_states):
                    #     row = self.state.Pi_a_s[s] * P_s[s]
                    #     pi_a_sums.append(row)
                    # pi_a = np.sum(np.asarray(pi_a_sums), axis=0)
                    pi_a = P_s@self.state.Pi_a_s
                    assert np.isclose(np.sum(pi_a), 1), f"Sum of Pi_a is not 1: {np.sum(pi_a)}"

                    #calculate Z
                    element_wise_a_by_q_table = pi_a * np.exp(self.state.beta * self.Q_table)
                    
                    zeta = np.sum((element_wise_a_by_q_table), axis = 1)
                    
                    #calculate for policy using partition fuction

                    new_Pi_a_s = (pi_a * np.exp(self.state.beta * self.Q_table))/ zeta.reshape(-1,1) 
                    self.state.Pi_a_s = new_Pi_a_s

                      # Frobenius norm for matrix difference
                    
                diff = np.linalg.norm(new_Pi_a_s - self.state.Pi_a_s, ord='fro')
                print(f"Epoch {epoch}: Frobenius norm difference: {diff:.4f}") # with percent
                #update state
                if next_state in goal_states:
                    print(f"Goal reached at state {next_state} in epoch {epoch} after {self.steps} steps.")
                    break
                # assert(False)
                current_state = next_state
                print(f"Epoch {epoch}: Step {self.steps}, Current state: {current_state}")
                self.steps += 1

               

      




        
    

In [10]:
env = SimpleEnv(render_mode="human")
env.reset()

({'image': array([[[1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [2, 5, 0],
          [2, 5, 0],
          [2, 5, 0],
          [2, 5, 0]],
  
         [[1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [1, 0, 0]],
  
         [[1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [1, 0, 0]],
  
         [[2, 5, 0],
          [2, 5, 0],
          [2, 5, 0],
          [2, 5, 0],
          [2, 5, 0],
          [2, 5, 0],
          [2, 5, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
 

In [11]:
info = blahut_witerations(env)

Free energy table shape: (256, 3)
policy shape: (256, 3)
s by s shape: (256, 256)


In [13]:
info.train()

current state position: (np.int64(8), np.int64(4)) current state direction 1, state index 125
Step 1: Action 1 taken.
next step position (np.int64(8), np.int64(4)), direction 2
next_state: 126
state t plus 1: 126
state t plus 1: 126
Convergence reached with diff_pias=0.004709845332405813
Step 2: Action 1 taken.
next step position (np.int64(8), np.int64(4)), direction 3
next_state: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus 1: 127
state t plus

KeyboardInterrupt: 

In [48]:
info.run_policy_2()

Step 0: State 224
Free energy values: [1.21118494e-121 1.00000000e+000 0.00000000e+000]
Chosen action: 1
Before step: Agent Pos: (1, 8), Dir: 0
After step: Agent Pos: (1, 8), Dir: 1
Step 1: State 225
Free energy values: [1.10790815e-124 1.00000000e+000 0.00000000e+000]
Chosen action: 1
Before step: Agent Pos: (1, 8), Dir: 1
After step: Agent Pos: (1, 8), Dir: 2
Step 2: State 226
Free energy values: [1.21118494e-121 1.00000000e+000 0.00000000e+000]
Chosen action: 1
Before step: Agent Pos: (1, 8), Dir: 2
After step: Agent Pos: (1, 8), Dir: 3
Step 3: State 227
Free energy values: [1.21118494e-121 1.00000000e+000 0.00000000e+000]
Chosen action: 1
Before step: Agent Pos: (1, 8), Dir: 3
After step: Agent Pos: (1, 8), Dir: 0
Step 4: State 224
Free energy values: [1.21118494e-121 1.00000000e+000 0.00000000e+000]
Chosen action: 1
Before step: Agent Pos: (1, 8), Dir: 0
After step: Agent Pos: (1, 8), Dir: 1
Step 5: State 225
Free energy values: [1.10790815e-124 1.00000000e+000 0.00000000e+000]
Ch

KeyboardInterrupt: 

In [96]:
np.exp(-121)

np.float64(2.820770088460135e-53)

In [85]:
info.Zeta

array([1.96096642e+13, 1.96096642e+13, 1.96096642e+13, 1.96096642e+13,
       1.96096642e+13, 1.96096642e+13, 1.96096642e+13, 1.96096642e+13,
       1.96096642e+13, 1.96096642e+13, 1.96096642e+13, 1.96096642e+13,
       1.96096642e+13, 1.96096642e+13, 1.96096642e+13, 1.96096642e+13,
       1.96096642e+13, 1.96096642e+13, 1.96096642e+13, 1.96096642e+13,
       1.96096642e+13, 1.96096642e+13, 1.96096642e+13, 1.96096642e+13,
       1.96096642e+13, 1.96096642e+13, 1.96096642e+13, 1.96096642e+13,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.13487151e-41, 1.96096642e+13, 1.96096642e+13, 1.96096642e+13,
       4.02022994e-34, 2.83303132e-38, 1.96096642e+13, 1.96096642e+13,
       9.49595572e-35, 1.96096642e+13, 1.96096642e+13, 1.96096642e+13,
       3.07532678e-34, 1.96096642e+13, 1.96096642e+13, 1.96096642e+13,
       4.23479413e-44, 1.96096642e+13, 1.96096642e+13, 1.96096642e+13,
       4.09136722e-44, 1.59074076e-34, 9.32351528e-44, 1.96096642e+13,
      

In [86]:
ones = np.where(info.Zeta == 1)

In [87]:
len(ones[0])

36

In [70]:
info.state_index_to_position(84)

(6, 3, 0)

In [88]:
info.free_energy_info # free energy, state, action, PI 

[(1,
  array([93.58886893, 93.48580222, 93.66076606]),
  np.int64(97),
  np.int64(2),
  array([6.26391610e-161, 1.00269257e-105, 1.00000000e+000])),
 (2,
  array([93.7379913 , 93.67973659, 93.72784678]),
  np.int64(129),
  np.int64(2),
  array([1.05563174e-116, 1.00269257e-105, 1.00000000e+000])),
 (3,
  array([93.7379913 , 93.67973659, 93.72784678]),
  np.int64(129),
  np.int64(2),
  array([2.60729006e-68, 2.47653635e-57, 1.00000000e+00])),
 (4,
  array([93.7379913 , 93.67973659, 93.72784678]),
  np.int64(129),
  np.int64(2),
  array([6.55218725e-20, 6.22359980e-09, 9.99999994e-01])),
 (5,
  array([93.7379913 , 93.67973659, 93.72784678]),
  np.int64(129),
  np.int64(2),
  array([1.00000000e+00, 1.05279701e-11, 1.85731510e-81])),
 (6,
  array([93.7379913 , 93.67973659, 93.72784678]),
  np.int64(129),
  np.int64(0),
  array([1.93646483e-068, 1.00000000e+000, 1.07040549e-100])),
 (7,
  array([93.75604493, 87.02534919, 87.0596066 ]),
  np.int64(128),
  np.int64(1),
  array([1.00000000e+00

In [26]:
info.Free_energy_table

array([[ -1.71263698,  -1.71263698,  -1.71263698],
       [ -1.71263698,  -1.71263698,  -1.71263698],
       [ -1.71263698,  -1.71263698,  -1.71263698],
       [ -1.71263698,  -1.71263698,  -1.71263698],
       [ -1.71263698,  -1.71263698,  -1.71263698],
       [ -1.71263698,  -1.71263698,  -1.71263698],
       [ -1.71263698,  -1.71263698,  -1.71263698],
       [ -1.71263698,  -1.71263698,  -1.71263698],
       [ -1.71263698,  -1.71263698,  -1.71263698],
       [ -1.71263698,  -1.71263698,  -1.71263698],
       [ -1.71263698,  -1.71263698,  -1.71263698],
       [ -1.71263698,  -1.71263698,  -1.71263698],
       [ -1.71263698,  -1.71263698,  -1.71263698],
       [ -1.71263698,  -1.71263698,  -1.71263698],
       [ -1.71263698,  -1.71263698,  -1.71263698],
       [ -1.71263698,  -1.71263698,  -1.71263698],
       [ -1.71263698,  -1.71263698,  -1.71263698],
       [ -1.71263698,  -1.71263698,  -1.71263698],
       [ -1.71263698,  -1.71263698,  -1.71263698],
       [ -1.71263698,  -1.71263

In [392]:
free_energy.P_s

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [391]:
indices = np.where(np.isnan(free_energy.P_s_by_s))
print(indices)


(array([61, 61, 61]), array([60, 61, 62]))


In [261]:
print(np.where(free_energy.P_s_by_s == 'nan'))

(array([], dtype=int64), array([], dtype=int64))


In [255]:
free_energy.Pi_a_s[215]

array([0.33333333, 0.33333333, 0.33333333])

In [81]:
free_energy.position_to_state_index((3,4,1))

np.int64(105)

In [595]:
env = SimpleEnv(render_mode=None)
env.reset()

({'image': array([[[1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [2, 5, 0],
          [2, 5, 0],
          [2, 5, 0],
          [2, 5, 0]],
  
         [[1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [1, 0, 0]],
  
         [[1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [1, 0, 0]],
  
         [[2, 5, 0],
          [2, 5, 0],
          [2, 5, 0],
          [2, 5, 0],
          [2, 5, 0],
          [2, 5, 0],
          [2, 5, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
 

In [543]:
free_energy = Qlearning_frompolicy(env)


Free energy table shape: (256, 3)
policy shape: (256, 3)
s by s shape: (256, 256)


In [596]:
info_to_go = blahut(env)

Free energy table shape: (256, 3)
policy shape: (256, 3)
s by s shape: (256, 256)


In [594]:
env_human = SimpleEnv(render_mode="human") #make same env but in human mode 

free_energy.env= env_human

In [97]:
free_energy.next_state_index(105, 2)

np.int64(137)

In [98]:
free_energy.state_index_to_position(137)

(3, 5, 1)

In [99]:
len(free_energy.allowed_state_idx)

220

In [597]:
info_to_go.train(10)

current state position: (np.int64(3), np.int64(7)) current state direction 2
Epoch 0: state: 202
Before step: 202, Pos: (np.int64(3), np.int64(7)), Dir: 2
Step 1: Action 0 taken.
next step position (np.int64(3), np.int64(7)), direction 1
next_state: 201
state t plus 1: 201
Before step: 201, Pos: (np.int64(3), np.int64(7)), Dir: 1
Step 2: Action 2 taken.
next step position (np.int64(3), np.int64(8)), direction 1
next_state: 233
state t plus 1: 233
Before step: 233, Pos: (np.int64(3), np.int64(8)), Dir: 1
Step 3: Action 1 taken.
next step position (np.int64(3), np.int64(8)), direction 2
next_state: 234
state t plus 1: 234
Before step: 234, Pos: (np.int64(3), np.int64(8)), Dir: 2
Step 4: Action 2 taken.
next step position (np.int64(2), np.int64(8)), direction 2
next_state: 230
state t plus 1: 230
Before step: 230, Pos: (np.int64(2), np.int64(8)), Dir: 2
Step 5: Action 2 taken.
next step position (np.int64(1), np.int64(8)), direction 2
next_state: 226
state t plus 1: 226
Before step: 226, 

In [598]:
#switch the render mode to human

env_human = SimpleEnv(render_mode="human") #make same env but in human mode 

info_to_go.env= env_human #switch out the env in q training with human env
info_to_go.run_policy_2()

Step 0: State 224
Free energy values: [9.02011843e-01 1.18727602e-13 9.79881572e-02]
Chosen action: 0
Before step: Agent Pos: (1, 8), Dir: 0
After step: Agent Pos: (1, 8), Dir: 3
Step 1: State 227
Free energy values: [1.68195713e-12 1.00000000e+00 1.77975426e-11]
Chosen action: 1
Before step: Agent Pos: (1, 8), Dir: 3
After step: Agent Pos: (1, 8), Dir: 0
Step 2: State 224
Free energy values: [9.02011843e-01 1.18727602e-13 9.79881572e-02]
Chosen action: 0
Before step: Agent Pos: (1, 8), Dir: 0
After step: Agent Pos: (1, 8), Dir: 3
Step 3: State 227
Free energy values: [1.68195713e-12 1.00000000e+00 1.77975426e-11]
Chosen action: 1
Before step: Agent Pos: (1, 8), Dir: 3
After step: Agent Pos: (1, 8), Dir: 0
Step 4: State 224
Free energy values: [9.02011843e-01 1.18727602e-13 9.79881572e-02]
Chosen action: 0
Before step: Agent Pos: (1, 8), Dir: 0
After step: Agent Pos: (1, 8), Dir: 3
Step 5: State 227
Free energy values: [1.68195713e-12 1.00000000e+00 1.77975426e-11]
Chosen action: 1
Bef

: 

In [485]:
np.where(np.isnan(info_to_go.Free_energy_table))

(array([], dtype=int64), array([], dtype=int64))

In [589]:
info_to_go.Free_energy_table

array([[-31.35866283, -24.17238105,   5.8305382 ],
       [-31.36028302, -50.42172592, -24.65376924],
       [-33.58632133,   1.59268347, -13.29100126],
       [-13.29100126,  -3.84968855,   5.27708845],
       [  5.46643454, -23.92808773, -26.53386588],
       [-25.64350561, -49.93537985, -39.50813923],
       [-37.33160301,   5.30378282, -29.94911227],
       [-24.4716988 ,   1.00155541, -25.64483477],
       [ -6.94783919, -10.95829982,  -7.12310322],
       [ 10.71495032,   2.05153434,   2.18706409],
       [ 20.75268732,  -1.1439739 , -13.29100538],
       [  5.72227083,  11.84937563, -26.53400499],
       [-21.08128449,  17.96274439, -23.20360121],
       [  5.57454619,  20.70172672, -11.95669774],
       [  0.98768374,   5.6733514 ,  10.08284384],
       [ -0.68286585,  -8.31438471,  17.39428032],
       [  6.64506751,  17.13843915, -11.87154573],
       [ -7.84101261,  -4.93183256, -27.3711991 ],
       [ -7.51139859, -13.23244927, -14.05425733],
       [  2.70703976,  17.04368

In [590]:
info_to_go.Pi_a

array([0.33333333, 0.33333333, 0.33333333])

In [591]:
info_to_go.Pi_a_s

array([[9.99243674e-01, 7.56325743e-04, 7.05678057e-17],
       [5.26890697e-09, 9.99999995e-01, 6.44345211e-12],
       [9.99999998e-01, 5.27172002e-16, 1.53410287e-09],
       [9.99920622e-01, 7.93698409e-05, 8.62872487e-09],
       [1.17897405e-14, 6.87674726e-02, 9.31232527e-01],
       [2.81942274e-11, 9.99970386e-01, 2.96137954e-05],
       [9.99378337e-01, 3.04380714e-19, 6.21662890e-04],
       [2.36288612e-01, 2.04432081e-12, 7.63711388e-01],
       [1.74325848e-02, 9.61795431e-01, 2.07719844e-02],
       [9.22337161e-05, 5.33781431e-01, 4.66126336e-01],
       [1.64062686e-15, 5.30406642e-06, 9.99994696e-01],
       [9.80115832e-15, 2.13948674e-17, 1.00000000e+00],
       [1.06946601e-01, 1.18184547e-18, 8.93053399e-01],
       [2.43375856e-08, 6.55581151e-15, 9.99999976e-01],
       [9.90747626e-01, 9.14120465e-03, 1.11169384e-04],
       [4.84688732e-04, 9.99515311e-01, 6.83373438e-12],
       [9.08525341e-09, 2.51839381e-13, 9.99999991e-01],
       [3.29721884e-09, 1.79765

In [482]:
info_to_go.Pi_a_s[251]

array([nan, nan, nan])

In [481]:
np.where(np.isnan(info_to_go.P_s_by_s))


(array([251, 251, 251]), array([219, 248, 250]))

In [479]:
info_to_go.P_s

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [462]:
info_to_go.Free_energy_table

array([[  0.        , 428.25377416, 107.23242865],
       [427.14531142,   0.        , 215.25728852],
       [417.93191065, 106.82472635,   0.        ],
       [417.82333548, 107.80230076, 419.45248244],
       [211.35306785, 109.00282692, 107.59461376],
       [  0.        , 108.10278772, 107.72199139],
       [107.83061774, 424.9329322 , 107.188864  ],
       [321.38111961, 215.87945019, 425.39109049],
       [106.82510046, 107.36896374, 213.38591716],
       [  0.        ,   0.        , 107.27973633],
       [  0.        , 323.05325747, 107.11530373],
       [215.41344289, 215.83998162, 323.05492254],
       [321.5969024 ,   0.        , 107.46857249],
       [  0.        ,   0.        , 107.70869022],
       [213.31538498, 107.42171287, 107.12418512],
       [214.94658445, 107.25832275, 107.99784805],
       [107.6031161 ,   0.        ,   0.        ],
       [  0.        , 108.65175493,   0.        ],
       [108.27419134,   0.        , 107.04434904],
       [107.86615608,   0.     

In [461]:
np.where(np.isnan(info_to_go.Free_energy_table))

(array([232]), array([2]))

In [460]:
np.where(np.isnan(info_to_go.Pi_a_s))

(array([232, 232, 232]), array([0, 1, 2]))

In [458]:
np.where(np.isnan(info_to_go.P_s_by_s))

(array([232, 232, 232]), array([233, 235, 236]))

In [456]:
info_to_go.P_s

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [ ]:
free_energy.train(40)

In [448]:
free_energy.Pi_a_s

array([[5.09434109e-46, 6.49474819e-01, 3.50525181e-01],
       [8.66527018e-47, 8.73599583e-01, 1.26400417e-01],
       [1.09152754e-01, 2.44890254e-46, 8.90847246e-01],
       [6.54083010e-02, 1.49441103e-01, 7.85150596e-01],
       [5.61758229e-46, 1.00000000e+00, 7.31803754e-46],
       [1.24608656e-46, 2.44598004e-01, 7.55401996e-01],
       [1.00000000e+00, 1.14784651e-45, 9.61228752e-47],
       [1.14923203e-01, 7.52325265e-02, 8.09844270e-01],
       [1.00000000e+00, 7.61647876e-47, 2.51832381e-47],
       [6.11546306e-01, 9.88603275e-47, 3.88453694e-01],
       [1.00000000e+00, 2.81017794e-45, 1.82769509e-45],
       [1.00000000e+00, 1.15474360e-45, 1.89872745e-45],
       [6.86424071e-02, 3.72644361e-01, 5.58713232e-01],
       [2.06868819e-47, 1.00000000e+00, 1.55773419e-46],
       [3.32846762e-01, 6.67153238e-01, 5.89749604e-47],
       [3.00798435e-01, 3.94667251e-01, 3.04534314e-01],
       [6.51462734e-47, 2.95560511e-01, 7.04439489e-01],
       [4.65657816e-01, 3.76411

In [430]:
free_energy.Free_energy_table

array([[ 5.67099119,  6.25176803,  5.95338109],
       [ 6.15514567, 11.37901093,  6.25421182],
       [ 7.16831335, 11.71401733,  5.55700365],
       [11.11271859,  6.74480764,  6.47759441],
       [ 6.66779172,  6.68405074, 12.86599413],
       [ 6.94510238,  6.7554723 ,  0.        ],
       [ 6.21523972, 12.64773375,  5.15517419],
       [11.47312435, 12.38008012, 11.79923317],
       [ 6.70484934,  7.20085764, 12.68649275],
       [ 5.99398679, 11.82913502,  7.18840662],
       [12.78949821, 12.40661602,  5.81865895],
       [12.33739386, 12.38322095,  6.68788075],
       [13.10687363,  6.97245448, 12.03243436],
       [ 7.30789769,  6.44971169,  7.56094211],
       [ 7.65532227,  6.9938978 ,  6.81564154],
       [12.57107191, 12.1878448 ,  6.97435202],
       [12.66850714,  6.55271124, 12.41698085],
       [12.7435444 ,  6.19623519,  5.90113518],
       [ 7.3816498 , 12.96631766, 12.4096121 ],
       [14.08098982, 12.6303862 , 12.96138655],
       [18.1501151 ,  6.85091309,  6.592

In [437]:
np.where(np.isnan(free_energy.Pi_a_s))

(array([27, 27, 27]), array([0, 1, 2]))

In [438]:
free_energy.Pi_a_s[25]

array([0.48270118, 0.00090734, 0.51639148])

In [436]:
np.where(np.isnan(free_energy.Zeta)) 

(array([27]),)

In [432]:
free_energy.Zeta

array([2.65468948e-03, 1.36479621e-03, 1.59866761e-03, 9.24122549e-04,
       8.22508217e-04, 3.49114357e-01, 2.66222229e-03, 7.37509849e-06,
       6.43129059e-04, 1.07770764e-03, 1.03760583e-03, 4.36897928e-04,
       3.08206214e-04, 9.14822070e-04, 8.35251299e-04, 3.28763062e-04,
       4.67233961e-04, 1.61821343e-03, 2.04948589e-04, 2.13431700e-06,
       8.22601053e-04, 3.48898318e-01, 4.76805211e-04, 3.02271363e-06,
       1.36971109e-03, 6.74722816e-01, 3.27626489e-01,            nan,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       3.27156756e-01, 3.27351308e-01, 2.81672143e-03, 2.47084091e-03,
       3.26871123e-01, 3.26506566e-01, 1.86170203e-03, 2.30574972e-03,
       6.87694928e-04, 7.12306828e-04, 1.33236608e-03, 2.15353823e-03,
       3.32482434e-06, 1.02829670e-03, 1.71097078e-03, 6.23062053e-04,
       1.02063946e-03, 2.90903782e-04, 9.86775263e-04, 3.26441298e-01,
       1.78567593e-03, 3.48990397e-01, 1.83069139e-03, 3.48983908e-01,
      

In [431]:
free_energy.Pi_a

array([nan, nan, nan])

In [418]:
np.where(free_energy.P_s_by_s == 'nan')

(array([], dtype=int64), array([], dtype=int64))

In [ ]:
np.where

In [416]:
free_energy.P_s

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [415]:
free_energy.Pi_a_s

array([[0.00000000e+000, 8.81451831e-257, 1.00000000e+000],
       [0.00000000e+000, 1.96779850e-271, 1.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 1.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 1.00000000e+000],
       [0.00000000e+000, 6.58701430e-256, 1.00000000e+000],
       [0.00000000e+000, 1.02491788e-294, 1.00000000e+000],
       [0.00000000e+000, 4.56421277e-247, 1.00000000e+000],
       [0.00000000e+000, 1.02491788e-294, 1.00000000e+000],
       [0.00000000e+000, 7.57290626e-263, 1.00000000e+000],
       [0.00000000e+000, 1.02491788e-294, 1.00000000e+000],
       [0.00000000e+000, 1.19352320e-262, 1.00000000e+000],
       [0.00000000e+000, 3.68563157e-258, 1.00000000e+000],
       [0.00000000e+000, 9.14065967e-256, 1.00000000e+000],
       [0.00000000e+000, 1.02491788e-294, 1.00000000e+000],
       [0.00000000e+000, 1.02491788e-294, 1.00000000e+000],
       [0.00000000e+000, 1.02491788e-294, 1.00000000e+000],
       [0.00000000e+000, 2.92565166e-249

In [390]:
free_energy.P_s

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [442]:
#switch the render mode to human

env_human = SimpleEnv(render_mode="human") #make same env but in human mode 

free_energy.env= env_human #switch out the env in q training with human env
free_energy.run_policy_2()

Step 0: State 224
Free energy values: [3.46281937e-46 8.57460090e-46 1.00000000e+00]
Chosen action: 2
Before step: Agent Pos: (1, 8), Dir: 0
After step: Agent Pos: (np.int64(2), np.int64(8)), Dir: 0
Step 1: State 228
Free energy values: [2.70302159e-01 1.50209055e-46 7.29697841e-01]
Chosen action: 2
Before step: Agent Pos: (np.int64(2), np.int64(8)), Dir: 0
After step: Agent Pos: (np.int64(3), np.int64(8)), Dir: 0
Step 2: State 232
Free energy values: [8.03533295e-01 3.28290582e-46 1.96466705e-01]
Chosen action: 0
Before step: Agent Pos: (np.int64(3), np.int64(8)), Dir: 0
After step: Agent Pos: (np.int64(3), np.int64(8)), Dir: 3
Step 3: State 235
Free energy values: [4.51672847e-47 1.00000000e+00 4.85605762e-46]
Chosen action: 1
Before step: Agent Pos: (np.int64(3), np.int64(8)), Dir: 3
After step: Agent Pos: (np.int64(3), np.int64(8)), Dir: 0
Step 4: State 232
Free energy values: [8.03533295e-01 3.28290582e-46 1.96466705e-01]
Chosen action: 0
Before step: Agent Pos: (np.int64(3), np.i

KeyboardInterrupt: 

# MLP Approach

In [ ]:

class MLP_approach:
    def __init__(
        self,
        env,    
    ):

        self.env = env
        self.num_states = np.int64(env.count_states())
        self.num_actions = env.action_space.n
        

        pi_net = nn.Sequential(
                    nn.Linear(self.num_states, 64),
                    nn.Tanh(),
                    nn.Linear(64, 64),
                    nn.Tanh(),
                    nn.Linear(64, self.num_actions)
                    )
        
        

# Kernel Approach

In [ ]:

env = SimpleEnv(render_mode="human")
free_energy_solver = FreeEnergyMin(env, beta=0.5)
# free_energy_solver.estimate_transitions() #for when you want to do a random walk, no learning
free_energy_solver.compute_free_energy()
free_energy_solver.run_policy()


In [ ]:
print("Action Space:", env.action_space)

In [ ]:
print("Action Space:", env.action_space)